### Declare Constants and Packages

In [ ]:
##==
const αₚₘ = 0.0081   # empirical constant used in calculation of Pierson-Moskowitz Spectrum
const αₖ =  0.012    # Charnock parameter or nondimensional roughness length
const g = 9.81      # Acceleration due to gravity (m/s^2)
const κ = 0.4       # Von Karman constant
const Ip = 2.5      # Thomson et al (2013) p.5957
const β = 0.0122    # empirical constant Juszko et al (1995) p.194, Thomson et al (2013) p.5954
#const z₀ = 0.002    # roughness length 
const γ = 3.3

using CSV, CurveFit
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using NativeFileDialog
using Plots, Printf
using Statistics, StatsBase
#using Tk

display(HTML("<style>.jp-Cell { width: 120% !important; }</style>"))

include("./Split_Spectra_Tools.jl")

### Read .HXV files for selected day

In [ ]:
# temporary code to handle spike in data for the Townsville record 25/01/2004 22:30               
function coarse_spike_filter(displacement)
##########################################
    
    xmean, xstdev = StatsBase.mean_and_std(displacement)
    suspects = findall(>(xmean + xstdev*3.5), abs.(displacement))
    [displacement[s] = 0.0 for s ∈ suspects]

    return(displacement)

end    # coarse_spike_filter()


# Calculate spread and direction from Fourier coefficients
function calc_spread_direction(a1, b1)
######################################
    
    θ₀ = atan.(b1,a1)
    m1 = (a1.^2 .+ b1.^2).^0.5
##    m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
##    n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

    # Calculate the spread
    σc = (2 .* (1 .- m1)).^0.5;

    direction = mod.(rad2deg.(atan.(b1,a1)),360)

    return(σc, direction)

end    # calc_spread_direction()


# finction to calculate f2 and Pden2 using Welch's algorithm
function calc_f2_Pden2(heave)
######################################
    
    # Present spectra using Welch's method to better define bimodal events
    ps_w = welch_pgram(heave, 256, 128; onesided=true, nfft=256, fs=sample_frequency, window=hanning);
    f2 = freq(ps_w);
    Pden2 = power(ps_w)

    return(f2, Pden2)

end    # calc_f2_Pden2()
    

function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i ∈ arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i ∈ arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
    end     # get_displacements()


function process_spectrum_file()
################################
    
    is_gps = false
    sync_word_location = findall(x -> x == "7FFF", df.Column2)

    for j ∈ sync_word_location

        try
            
            i = df.Column2[j+1]
            word_number = parse(Int, SubString.(i, 1, 1), base=16)*16^0
            word = parse(Int, SubString.(i, 2, 2), base=16)*16^2 + parse(Int, SubString.(i, 3, 3), base=16)*16^1 + parse(Int, SubString.(i, 4, 4), base=16)*16^0
##            println(j,' ',word_number,' ',word)
    
            # Test whether buoy is MkIII or DWR-G - see p.51 Table 5.7.5a. Organization and significance of the system file data 
            # If DWR-G:
            #     Av0 = 0; Ax0 = 0; Ay0 = 0; O = 0; and Inclination = 0
            
            if (word_number == 7 && word == 0)
                is_gps = true
            end

        catch

            #do nothing

        end
            
    end
    
    return(is_gps)
    
    end    # process_spectrum_file()


function fix_gps_errors(heave)
# function to apply polynomial fit to WSE's affected by GPS errors
# uses selectable offset value to fine-tune result
#####################################    
    
    # locate GPS errors
    gps_errors = findall(isodd,parse.(Int,SubString.(string.(df.Column4), 2, 2), base = 16))
    
    if isempty(gps_errors)
        
        println("") #println("No GPS errors in this record")
        
    else
        
        println(length(gps_errors)," GPS errors")
        
        for ii ∈ reverse(gps_errors)

            error_center = ii

            # User-selected offset either side of GPS error
            lower_offset = upper_offset = 120

            if error_center <= lower_offset
                lower_offset = error_center - 1
            end

            if error_center+upper_offset > 2304
                upper_offset = 2304 - error_center
            end

            # Fit curve to subset of heave before GPS error
            left_side_points = error_center-lower_offset:error_center
            fit1 = curve_fit(Polynomial, left_side_points, heave[left_side_points], 2)
            yfit1 = fit1.(left_side_points)
            yfit1[length(yfit1)] = 0.0

            # Fit curve to subset of heave after GPS error
            right_side_points = error_center:error_center+upper_offset
            fit2 = curve_fit(Polynomial, right_side_points, heave[right_side_points], 2)
            yfit2 = fit2.(right_side_points)
            yfit2[1] = 0.0

            # apply polynomial results to wse's on both sides of GPS error
            heave[left_side_points] .= heave[left_side_points] - yfit1
            heave[right_side_points] .= heave[right_side_points] - yfit2
            heave[ii] = 0.0    # set wse at GPS error location to 0

        end
    
    end
    
    return(heave)
    
    end     # fix_gps_errors()


# extract the displacements from the HEX data
function get_HNW(infil)
#####################################
        
    df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));

    # Remove rows that do not match Datawell's data format
    df = df[Bool[length(x) == 24 for x in df.Column1], :]


    is_gps = process_spectrum_file()
##    println(is_gps)
    
    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    sequence = []

    for i ∈ arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    # Calculate heave WSEs
    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    if is_gps

        heave = fix_gps_errors(heave)

    end

    return(heave, north, west)

    end    # get_HNW()


#######################################################################################
#######################################################################################
#######################################################################################    

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
const sample_frequency = 1.28

hxv_directory = pick_folder() * "\\"

# build list of all hxv files in selected directory
hxv_files = filter(x->occursin(".hxv",x), readdir(hxv_directory));
hxv_files = hxv_files[findall(x->endswith(uppercase(x), ".HXV"), hxv_files)];

nyquist = sample_frequency/2

# build df containing displacements and Fourier coefficient for selected day
displacement_df = DataFrame(Date = [], Heave = [], North = [], West = [], fhh = [], Chh = [], 
    a1 = [], b1 = [], a2 = [], b2 = [], f2 = [], Pden2 = [], Spread = [], Direction = [])

println("Reading files:")
flush(stdout)

for infil ∈ hxv_files
    
    date_string = split(split(infil,"_")[2],".")[1]
    date = Dates.DateTime.(date_string, "yyy-mm-ddTHHhMMK")

    print(date_string*" ")
        
    heave, north, west = get_HNW(hxv_directory * infil)
            
########################################################################### 
    heave = coarse_spike_filter(heave)
    north = coarse_spike_filter(north)
    west = coarse_spike_filter(west)
###########################################################################
            
    fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)
    
    σc, direction = calc_spread_direction(a1, b1)

    f2, Pden2 = calc_f2_Pden2(heave)

    push!(displacement_df, (date, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))
    
end

println("Done!")  

### Read a month of .HXV files

In [ ]:
using CSV
using Dates, DataFrames
using Glob
using Dates, DataFrames, Distributions, DSP
using LaTeXStrings
using Printf
using Statistics #, StatsPlotss
using StatsBase

# Function for splitting filename and extracting details
function extract_details(infil)
###############################
    
    split_details = split(split(infil, "\\")[end], ".")[1]
    site_name = split_details[1:4]
    date_string = split_details[6:15]
    time_string = replace(split_details[17:21], 'h' => ':')
    date = DateTime(date_string * " " * time_string, "yyyy-mm-dd HH:MM")
    date_string*" "*time_string

    return(site_name, date, date_string*" "*time_string)
    
end    # extract_details()


#######################################################################################################
#######################################################################################################
#######################################################################################################

# Widen screen for better viewing
display("text/html", "<style>.container { width:120% !important; }</style>")

const sample_frequency = 1.28

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .HXV files in the directory and subdirectories
hxv_files = glob("**/*.hxv", directory_path)

# build df containing displacements and Fourier coefficient for selected day
global displacement_df = DataFrame(Date = [], Heave = [], North = [], West = [], fhh = [], Chh = [], 
    a1 = [], b1 = [], a2 = [], b2 = [], f2 = [], Pden2 = [], Spread = [], Direction = [])

println("Reading all .HXV files for selected month (this takes a while!):")
flush(stdout)

for infil ∈ hxv_files
    
    site_name, date, date_string = extract_details(infil)

    print(date_string*" ")
        
    heave, north, west = get_HNW(infil)
            
########################################################################### 
    heave = coarse_spike_filter(heave)
    north = coarse_spike_filter(north)
    west = coarse_spike_filter(west)
###########################################################################
            
    # get frequencies and spectra, and Fourier coefficients
    fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)   
    f2, Pden2 = calc_f2_Pden2(heave)

    # get spread and direction
    σc, direction = calc_spread_direction(a1, b1)

    push!(displacement_df, (date, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))

end

println("Done!")

### Plot spectrogram

In [ ]:
times = displacement_df.Date
frequency_range = displacement_df.f2[1]
spectral_data = hcat(displacement_df.Pden2...)'
max_spec = maximum(spectral_data)

start_date = first(displacement_df.Date)
last_date = last(displacement_df.Date)

# display plots to screen
tm_tick = range(first(displacement_df.Date),last(displacement_df.Date),step=Day(1))
ticks = Dates.format.(tm_tick,"dd")

##p1 = heatmap(times, frequency_range, spectral_data,
##        xlabel="Time", ylabel="Frequency (Hz)", title="Spectrogram", lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0.0,max_spec), levels=10, fill=true)
p1 = Plots.contourf(times, frequency_range, spectral_data', lw=0.5, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)
#p1 = Plots.contour!(times, frequency_range, spectral_data', lw=0.25, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10)

# draw grid lines on plot
for i ∈ 0:0.1:0.6
    hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i ∈ start_date:Day(1):last_date
    vline!(p1, [i], lw=0.5, c=:white, label="")
end

site = uppercase(split(split(hxv_files[1],"\\")[end],"_")[1])

title = site*" "*Dates.format(start_date, "yyyy-mm-dd HH:MM")*" to "*Dates.format(last_date, "yyyy-mm-dd HH:MM")

Plots.plot(p1, xlabel="Time", xlim=(start_date,last_date), xticks=(tm_tick,ticks), xtickfontsize=7,
    ylabel="Frequency (Hz)", ylim=(0,0.4), ytickfontsize=8, 
    title=title, framestyle = :box,
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,800), gridlinewidth=0.5, 
    gridstyle=:dot, gridalpha=1, colorbar=true, colorbar_title="Spectral energy (m²/Hz.)")

### Test code to calc wind from waves

In [ ]:
using CSV, Dates, DataFrames, Glob, DSP, Statistics
using NativeFileDialog

const sample_frequency = 1.28

# Define the path to the directory you want to search in
directory_path = pick_folder()

# Use glob to find all .HXV files in the directory and subdirectories
hxv_files = glob("**/*.hxv", directory_path)

# Preallocate DataFrame
num_files = length(hxv_files)
displacement_df = DataFrame(Date = DateTime[], Heave = [], North = [], West = [],
                            fhh = [], Chh = [], a1 = [], b1 = [], a2 = [], b2 = [],
                            f2 = [], Pden2 = [], Spread = [], Direction = [])

println("Reading all .HXV files for selected month (this takes a while!):")
flush(stdout)

# Function for spike filtering
function coarse_spike_filter!(displacement)
    xmean, xstdev = mean_and_std(displacement)
    suspects = findall(>(xmean + xstdev * 3.5), abs.(displacement))
    displacement[suspects] .= 0.0
    return displacement
end


# Process each file
@time begin
        
    for infil ∈ hxv_files
        
        site_name, date, date_string = extract_details(infil)
    
        print(date_string*" ")
            
        heave, north, west = get_HNW(infil)
                
########################################################################### 
        heave = coarse_spike_filter(heave)
        north = coarse_spike_filter(north)
        west = coarse_spike_filter(west)
###########################################################################
                
        # get frequencies and spectra, and Fourier coefficients
        fhh, Chh, a1, b1, a2, b2 = get_Fourier_coefficients(heave, north, west)   
        f2, Pden2 = calc_f2_Pden2(heave)
    
        # get spread and direction
        σc, direction = calc_spread_direction(a1, b1)
    
        push!(displacement_df, (date, heave, north, west, fhh, Chh, a1, b1, a2, b2, f2, Pden2, σc, direction))

    end
    
end

println("Done!")


In [ ]:
#****************************************************************************************************
const αₚₘ = 0.0081
const αₖ = 0.012     # Charnock constant
const g = 9.81      # Acceleration due to gravity (m/s^2)
const κ = 0.4       # Von Karman constant - Thomson et al (2013) p.5957
const Ip = 2.5      # Thomson et al (2013) p.5957
const β = 0.0122    # empirical constant Juszko et al (1995) p.194             , Thomson et al (2013) p.5954
##const z₀ = 0.002    # roughness length 
const γ = 3.3
#****************************************************************************************************
z = 10       # Desired anemometer height at wave site

using GLM
using LsqFit
using LinearAlgebra
using Peaks
using Printf
using Roots
using StatsBase

# find closest frequency in arr to target value
function search_nearest_index(arr, target)
##########################################
    
    differences = abs.(arr .- target)
    nearest_index = argmin(differences)
    
    return(nearest_index)

end    # search_nearest_index()


# use a nearest neighbour approach to locate the closest point
function nearest_neighbor(target, points)
#########################################
    
    distances = [norm(target - point) for point ∈ points]
    nearest_index = argmin(distances)
    
    return(points[nearest_index])
    
end    # nearest_neighbor()


# Helper function to filter peaks based on a cutoff value
## called by find_peaks_and_valleys()
function filter_peaks(peak_freqs, peak_values, cutoff)
    
    filtered_indices = findall(x -> x >= cutoff, peak_values)
    
    return(peak_freqs[filtered_indices], peak_values[filtered_indices])
    
end    # filter_peaks()


# Helper function to merge nearby peaks within a specified tolerance
## called by find_peaks_and_valleys()
function merge_peaks(filtered_peak_freqs, filtered_peak_values, tolerance)
##########################################################################
    
    merged_freqs = Float64[]; merged_values = Float64[]
    
    for i in 1:length(filtered_peak_freqs)
        
        if isempty(merged_freqs) || minimum(abs.(filtered_peak_freqs[i] .- merged_freqs)) > tolerance
            
            push!(merged_freqs, filtered_peak_freqs[i])
            push!(merged_values, filtered_peak_values[i])
            
        else
            
            for j in 1:length(merged_freqs)
                if abs(filtered_peak_freqs[i] - merged_freqs[j]) <= tolerance
                    if filtered_peak_values[i] > merged_values[j]
                        merged_freqs[j] = filtered_peak_freqs[i]
                        merged_values[j] = filtered_peak_values[i]
                    end
                end
            end
            
        end
        
    end
    
    return(merged_freqs, merged_values)
    
end    # merge_peaks()


# function to find valleys between merged peaks
## called by find_peaks_and_valleys()
function find_valleys(freqs, spectrum, merged_freqs)
#################################################### 
    
    separation_freqs = Float64[]; separation_values = Float64[]
    
    for i in 2:length(merged_freqs)
        
        interval_idx = findall(x -> x >= merged_freqs[i-1] && x <= merged_freqs[i], freqs)
        if !isempty(interval_idx)
            interval_spectrum = spectrum[interval_idx]
            minima = findminima(interval_spectrum)
            if !isempty(minima[1])
                min_val_index = argmin(minima[2])
                valley_idx = interval_idx[minima[1][min_val_index]]
                push!(separation_freqs, freqs[valley_idx])
                push!(separation_values, minima[2][min_val_index])
            end
        end
        
    end
    
    return(separation_freqs, separation_values)
    
end    # find_valleys()


"""
    find_peaks_and_valleys(freqs::Vector{Float64}, spectrum::Vector{Float64}, peak_vals::Vector{Float64})

Finds the peaks and valleys in the given spectral data.

# Arguments
- `freqs::Vector{Float64}`: Array of frequencies.
- `spectrum::Vector{Float64}`: Array of spectral values corresponding to the frequencies.
- `peak_vals::Vector{Float64}`: Array of peak values from the `findmaxima` function.

# Returns
- `merged_freqs::Vector{Float64}`: Frequencies of the merged peaks.
- `merged_values::Vector{Float64}`: Values of the merged peaks.
- `separation_freqs::Vector{Float64}`: Frequencies of the separation valleys.
- `separation_values::Vector{Float64}`: Values of the separation valleys.
- `cutoff::Float64`: Cutoff value used for filtering peaks.
"""
function find_peaks_and_valleys(freqs::Vector{Float64}, spectrum::Vector{Float64}, peak_vals::Vector{Float64})
#################################################################################################################    
    maxima = findmaxima(spectrum)
    peak_freqs, peak_values = freqs[maxima[1]], maxima[2]

    max_peak = maximum(peak_vals)
    cutoff = max_peak * 0.2

    filtered_peak_freqs, filtered_peak_values = filter_peaks(peak_freqs, peak_values, cutoff)
    merged_freqs, merged_values = merge_peaks(filtered_peak_freqs, filtered_peak_values, 0.05)
    separation_freqs, separation_values = find_valleys(freqs, spectrum, merged_freqs)

    return merged_freqs, merged_values, separation_freqs, separation_values, cutoff
end    # find_peaks_and_valleys()


# function to calc spectral moments
function calc_moments(t, y)
##########################
    
"""
    Calls: Nil
    Called by: calc_PM_JONSWAP()
"""
    ax1 = (last(t) - first(t)) / (length(t)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s_₀₁, s₀₀, s₀₁, s₀₂, s₀₃, s₀₄ = 0, 0, 0, 0, 0, 0
    m₋₁, m₀, m₁, m₂, m₃, m₄ = 0, 0, 0, 0, 0, 0

    for ii ∈ 1:length(t)

        s_₀₁ += t[ii]^-1 * y[ii]
        s₀₀ += t[ii]^0 * y[ii]
        s₀₁ += t[ii]^1 * y[ii]
        s₀₂ += t[ii]^2 * y[ii]
        s₀₃ += t[ii]^3 * y[ii]
        s₀₄ += t[ii]^4 * y[ii]

    end

    m₋₁ = 0.5*ax1*(first(t)^-1*first(y) + 2*s_₀₁ + last(t)^0*last(y))
    m₀ = 0.5*ax1*(first(t)^0*first(y) + 2*s₀₀ + last(t)^0*last(y))
    m₁ = 0.5*ax1*(first(t)^1*first(y) + 2*s₀₁ + last(t)^1*last(y))
    m₂ = 0.5*ax1*(first(t)^2*first(y) + 2*s₀₂ + last(t)^2*last(y))
    m₃ = 0.5*ax1*(first(t)^3*first(y) + 2*s₀₃ + last(t)^3*last(y))
    m₄ = 0.5*ax1*(first(t)^4*first(y) + 2*s₀₄ + last(t)^4*last(y))        

    return(m₋₁, m₀, m₁, m₂, m₄)

    end    # calc_moments()


# calculate frequency domain parameters
function calc_parameters(t_,y_)
###############################

"""
    Calls: calc_moments
"""
            
    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t_,y_)
    hₘ₀ = 4√m₀        # significant wave height
    Hᵣₘₛ = √(8m₀)      # root mean square wave height
    T₀₂ = √(m₀/m₂)    # mean wave period
    T₀₁ = m₀/m₁       # significant wave period
    T₋₁₀ = m₋₁/m₀     # mean energy period Tₑ
    Tc = √(m₂/m₄)     # crest period
            
    return(hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc)

    end    # calc_parameters()


# identify whether this part of the spectra is wind sea or ocean swell
function get_wave_types(start, next, t, y, dir, previous_wave_type, separation_point)
#########################################################################################

"""
    Calls: calc_parameters
"""
    t = t[start:next]
    y = y[start:next]
    dir = dir[start:next]

    peak = argmax(y)
    fₚ = t[peak]
    Tₚ = 1/fₚ  
            
    hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc = calc_parameters(t,y)         
    
    # get peak direction and mean direction (weighted by spectral energy values)
    peak_direction = dir[peak]
    weighted_mean_direction = mean(dir, Weights(y))

    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t]

    # determine whether this part of wave record is sea or swell
    # Refer https://www.researchgate.net/publication/249605099_Spectral_Partitioning_and_Identification_of_Wind_Sea_and_Swell
    ratio = y[peak] / Sf[peak]
    wave_type = (ratio <= 1 ? "Ocean Swell" : "Wind Sea")

    # test whether separation between sea and swell has been found
    if (previous_wave_type == "Ocean Swell") && (wave_type == "Wind Sea")
    # Separation between swell and sea has been located
        separation_point = start
    end

    previous_wave_type = wave_type

    return(previous_wave_type, separation_point)
        
    end    # get_wave_types()


# calculate best the linear regression slope of segments
function calc_best_slope(ff_subset, ss_subset)
##############################################    
    
    # Initialize an array to store the slopes
    slopes = []
    ranges = Tuple{Int, Int}[] 

    for i ∈ 1:length(ss_subset)-15

        j = i+14

    ##    println("$i $j")
        # Select the segment of the arrays
        ss_segment = ss_subset[i:j]
        ff_segment = ff_subset[i:j]

        # Normalize ss_segment by multiplying it by ff_segment to the fourth power
        normalized_ss_segment = ss_segment .* ff_segment.^4

        # Calculate the slope of the linear regression
        slope = cov(normalized_ss_segment, ff_segment) / var(ff_segment)

        # Store the slope
        push!(slopes, slope)
        push!(ranges,(i,j))

    end

    # find the range for the best slope
    range = ranges[argmin(abs.(slopes))]
        
    return(range)    
            
end    # calc_best_slope()
 
            
######################################################################################################
######################################################################################################
######################################################################################################

date_val = []
wind_val = []
dir_val = []

for iii ∈ 1:nrow(displacement_df)
    date = Dates.format.(displacement_df.Date[iii],"yyyy-mm-dd HH:MM")
    sea_swell_transition = 1
    
    date = Dates.format( displacement_df.Date[iii], "yyyy-mm-dd HH:MM")
    t = displacement_df.fhh[iii]
    y = displacement_df.Chh[iii]

    dir = displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 0.6)
    t = t[valid]
    y = y[valid]
    dir = dir[valid]
    spread = spread[valid]

    # locate ALL peaks
    peaks, peak_vals = findmaxima(y)

    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
##    println(length(merged_freqs) == 1 ? "Unimodal spectra" : "Bimodal spectra")

    # Initial starting point
    start = 1

    # assume we are starting with an "ocean swell"
    previous_wave_type = "Wind Sea"
    global separation_point = 0

    # repeat for each separation frequency detected
    for ii ∈ separation_freqs

        next = findfirst(x->x==ii, t)
        previous_wave_type, separation_point = get_wave_types(start, next, t, y, dir, previous_wave_type, separation_point)
        start = next

    end

    # Do the final wave type
    next = findfirst(x->x==t[end], t)
    previous_wave_type, separation_point = get_wave_types(start, next, t, y, dir, previous_wave_type, separation_point)

    
    if separation_point != 0

        sea_swell_transition = separation_point

    end

    # get spectral peak of wind sea and find start of equilibrium and saturation ranges
    wind_sea_peak = sea_swell_transition + argmax(y[sea_swell_transition:end]) - 1   
    start_equilibrium = search_nearest_index(t, t[wind_sea_peak] * 1.3)    # from Voermans et al (2020) p.2 citing Toba (1973)
    start_saturation = search_nearest_index(t, t[wind_sea_peak] * 3.0)
                                   
    frequency = t[start_equilibrium:start_saturation]
    spectra = y[start_equilibrium:start_saturation]
    direction = dir[start_equilibrium:start_saturation]

    try
        
        if sea_swell_transition != 1  
    
    #        start_saturation = max(0.6, start_saturation)
            # processing mixed sea-swell spectra
            ranges = calc_best_slope(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])                 
            
        else
    
            # processing swell spectra
            if previous_wave_type == "Ocean Swell"
            
                swell_peak = argmax(y) # find spectral peak
                start_equilibrium = search_nearest_index(t, t[swell_peak] * 2)    # see Mudd et al (2024) p3.
                ranges = calc_best_slope(t[start_equilibrium:end], y[start_equilibrium:end])                 
            
            else
    
                #processing sea spectra
                ranges = calc_best_slope(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])                 
                
            end
                
        end
    
        best_range = mean(y[start_equilibrium .+ ranges[1] .- 1:start_equilibrium .+ ranges[2] .- 1] .* t[start_equilibrium .+ ranges[1] .- 1:start_equilibrium .+ ranges[2] .- 1].^4)
        best_dir =  mean(dir[start_equilibrium .+ ranges[1] .- 1:start_equilibrium .+ ranges[2] .- 1])
        best_fit_start = start_equilibrium .+ ranges[1] .- 1            
        best_fit_end = start_equilibrium .+ ranges[2] .- 1  
    
        u_star = best_range * (2π)^3 / (4*β*Ip*g)    # Mudd et. al. (2024) Equation (4) p.3 - rearrangement of Thomson et. al's, (2013) Equation (2)
    
        # Estimate roughness length (z₀) using Charnock relation
        z₀ = αₖ * u_star^2 / g    # Charnock relation - from Thomson et. al. (2013) Equation (5) p.2952
        
        # Calculate vertical profile of horizontal wind velocity (Uz)
        u10 = mean([u_star / κ * log(z / z₀)]) # Thomson et. al. (2013) Equation (4) p.2952
        @printf("%s U₁₀ estimate = %5.2fm/s (%5.2fkm/hr)\n",date, u10, u10*3.6)
     
        push!(date_val, displacement_df.Date[iii])
        push!(wind_val, u10 * 3.6)    # calculated U10 wind speed in km/hr
        push!(dir_val, best_dir)
    catch
        println("Data error at ",string(displacement_df.Date[iii]))
    end
end

In [ ]:
using Interpolations

# Helper function to calculate the Pierson-Moskowitz spectrum
function pierson_moskowitz_spectrum(freqs, f_p, alpha=0.0081, g=9.81)
    S_f = alpha * g^2 * (2 * pi)^(-4) .* freqs.^(-5) .* exp.(-5/4 * (f_p ./ freqs).^4)
    return S_f
end

# Function to classify peaks as sea or swell
function classify_peaks(merged_freqs, merged_values, f_p, spectrum, freqs)
    # Calculate PM spectrum
    pm_spectrum = pierson_moskowitz_spectrum(freqs, f_p)

    # Interpolate PM spectrum to the frequencies of the merged peaks
    pm_at_peak_freqs = interp(freqs, pm_spectrum, merged_freqs)
    
    # Classify peaks
    swell_peaks = findall(merged_values .< pm_at_peak_freqs)
    sea_peaks = findall(merged_values .>= pm_at_peak_freqs)

    return merged_freqs[swell_peaks], merged_values[swell_peaks], merged_freqs[sea_peaks], merged_values[sea_peaks]
end

# Complete workflow function
function find_peaks_valleys_and_classify(t, y, peak_vals, f_p)
    # Find peaks and valleys
    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
    
    # Classify peaks
    swell_freqs, swell_values, sea_freqs, sea_values = classify_peaks(merged_freqs, merged_values, f_p, y, t)
    
    return swell_freqs, swell_values, sea_freqs, sea_values, separation_freqs, separation_values, cutoff
end

iii = 1
t = displacement_df.fhh[iii]
y = displacement_df.Chh[iii]
dir = displacement_df.Direction[iii]
spread = displacement_df.Spread[iii]

# only use data in the frequency range 0.03 - 0.6 Hz.
valid = findall(0.03 .< t .< 0.6)
t = t[valid]
y = y[valid]
dir = dir[valid]
spread = spread[valid]

using Peaks

# Find the maxima in the spectrum
peaks, peak_vals = findmaxima(y)
maxima = findmaxima(y)
peak_vals = maxima[2] # Peak values from the spectrum

# Assume f_p is known or calculated from the data
f_p = 0.1 # Peak frequency for the Pierson-Moskowitz spectrum

# Helper function to calculate the Pierson-Moskowitz spectrum
function pierson_moskowitz_spectrum(freqs, f_p, alpha=0.0081, g=9.81)
    S_f = alpha * g^2 * (2 * pi)^(-4) .* freqs.^(-5) .* exp.(-5/4 * (f_p ./ freqs).^4)
    return S_f
end

# Function to classify peaks as sea or swell
function classify_peaks(merged_freqs, merged_values, f_p, spectrum, freqs)
    # Calculate PM spectrum
    pm_spectrum = pierson_moskowitz_spectrum(freqs, f_p)

    # Interpolate PM spectrum to the frequencies of the merged peaks
    itp = interpolate(freqs, pm_spectrum, Gridded(Linear()))
    pm_at_peak_freqs = itp.(merged_freqs)
    
    # Classify peaks
    swell_peaks = findall(merged_values .< pm_at_peak_freqs)
    sea_peaks = findall(merged_values .>= pm_at_peak_freqs)

    return merged_freqs[swell_peaks], merged_values[swell_peaks], merged_freqs[sea_peaks], merged_values[sea_peaks]
end

# Complete workflow function
function find_peaks_valleys_and_classify(t, y, peak_vals, f_p)
    # Find peaks and valleys
    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
    
    # Classify peaks
    swell_freqs, swell_values, sea_freqs, sea_values = classify_peaks(merged_freqs, merged_values, f_p, y, t)
    
    return swell_freqs, swell_values, sea_freqs, sea_values, separation_freqs, separation_values, cutoff
end

# Using the find_peaks_valleys_and_classify function
swell_freqs, swell_values, sea_freqs, sea_values, separation_freqs, separation_values, cutoff = find_peaks_valleys_and_classify(t, y, peak_vals, f_p)

# Print results
println("Swell Peaks: Frequencies: $swell_freqs, Values: $swell_values")
println("Sea Peaks: Frequencies: $sea_freqs, Values: $sea_values")


In [ ]:
using Peaks
using Interpolations

# Example spectral data (replace with your actual data)
freqency = t # Frequency array
spectrum = y # Example spectral data
peaks, peak_vals
# Find the maxima in the spectrum
maxima = findmaxima(spectrum)
peak_vals = maxima[2] # Peak values from the spectrum

# Assume f_p is known or calculated from the data
f_p = 0.1 # Peak frequency for the Pierson-Moskowitz spectrum

# Helper function to calculate the Pierson-Moskowitz spectrum
function pierson_moskowitz_spectrum(freqency, f_p, alpha=0.0081, g=9.81)
    S_f = alpha * g^2 * (2 * pi)^(-4) .* freqency.^(-5) .* exp.(-5/4 * (f_p ./ freqency).^4)
    return S_f
end

# Function to classify peaks as sea or swell
function classify_peaks(merged_freqency, merged_values, f_p, spectrum, freqency)
    # Calculate PM spectrum
    pm_spectrum = pierson_moskowitz_spectrum(freqency, f_p)

    # Create the interpolation object
    itp = LinearInterpolation(freqency, pm_spectrum, extrapolation_bc=Flat())
    pm_at_peak_freqency = itp.(merged_freqency)
    
    # Classify peaks
    swell_peaks = findall(merged_values .< pm_at_peak_freqency)
    sea_peaks = findall(merged_values .>= pm_at_peak_freqency)

    # Find the maximum energy peak for swell and sea
    if !isempty(swell_peaks)
        max_swell_index = argmax(merged_values[swell_peaks])
        swell_freq = merged_freqency[swell_peaks[max_swell_index]]
        swell_value = merged_values[swell_peaks[max_swell_index]]
    else
        swell_freq, swell_value = NaN, NaN
    end

    if !isempty(sea_peaks)
        max_sea_index = argmax(merged_values[sea_peaks])
        sea_freq = merged_freqency[sea_peaks[max_sea_index]]
        sea_value = merged_values[sea_peaks[max_sea_index]]
    else
        sea_freq, sea_value = NaN, NaN
    end

    return swell_freq, swell_value, sea_freq, sea_value
end

# Complete workflow function
function find_peaks_valleys_and_classify(t, y, peak_vals, f_p)
    # Find peaks and valleys
    merged_freqency, merged_values, separation_freqency, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
    
    # Classify peaks
    swell_freq, swell_value, sea_freq, sea_value = classify_peaks(merged_freqency, merged_values, f_p, y, t)
    
    return swell_freq, swell_value, sea_freq, sea_value, separation_freqency, separation_values, cutoff
end

# Using the find_peaks_valleys_and_classify function
swell_freq, swell_value, sea_freq, sea_value, separation_freqency, separation_values, cutoff = find_peaks_valleys_and_classify(freqency, spectrum, filtered_peak_values, f_p)

# Print results
println("Swell Peak: Frequency: $swell_freq, Value: $swell_value")
println("Sea Peak: Frequency: $sea_freq, Value: $sea_value")


In [ ]:
# Define the date you are interested in
target_date = Date("2009-05-25")

# Filter the DataFrame for the target date
result_df = filter(row -> target_date < row.Date < target_date + Day(1) , displacement_df)


In [ ]:
display(HTML("<style>.jp-Cell { width: 120% !important; }</style>"))

# Helper function to calculate the Pierson-Moskowitz spectrum
function pierson_moskowitz_spectrum(freqency, f_p, alpha=0.0081, g=9.81)
    
    S_f = alpha * g^2 * (2 * pi)^(-4) .* freqency.^(-4) .* exp.(-5/4 * (f_p ./ freqency).^4)
    S_f = replace(S_f, NaN => 0.0)
    
    return(S_f)
    
end    # pierson_moskowitz_spectrum()

function filter_peaks(peak_freqs, peak_values, cutoff)
    
    filtered_indices = findall(x -> x >= cutoff, peak_values)
    
    return(peak_freqs[filtered_indices], peak_values[filtered_indices])
    
end    # filter_peaks()

iii = 24
t = result_df.fhh[iii]
y = result_df.Chh[iii]
dir = result_df.Direction[iii]
spread = result_df.Spread[iii]

peaks, peak_vals = findmaxima(y)
cutoff = maximum(peak_vals) * 0.075

filtered_peak_freqs, filtered_peak_values = filter_peaks(t[peaks], peak_vals, cutoff)

p1 = Plots.plot(t, y, xminorticks=10, lc=:red, lw=:2, ylims=(0,4), size=(1600,600))
p1 = Plots.scatter!(filtered_peak_freqs, filtered_peak_values)
p1 = Plots.hline!([cutoff],label="")

for ii in 1:length(filtered_peak_freqs)
    
    pm_spectrum = pierson_moskowitz_spectrum(t, filtered_peak_freqs[ii])
    max_PM = maximum(pm_spectrum)
    if max_PM >= filtered_peak_values[ii]

        @printf("%5.4f %5.4f %5.4f  is Ocean Swell\n",1/filtered_peak_freqs[ii], filtered_peak_values[ii], max_PM)

    else

        @printf("%5.4f %5.4f %5.4f  is Wind Sea\n",1/filtered_peak_freqs[ii], filtered_peak_values[ii], max_PM)


    end
    p1 = Plots.plot!(t,pierson_moskowitz_spectrum(t, filtered_peak_freqs[ii]), label="")
    subplot = Plots.twinx()
    p1 = Plots.plot!(subplot, t, dir, label="")
end

display(p1)

In [ ]:
using Interpolations, Plots

# Lower and higher bound of interval
a = 1
b = 100
# Interval definition
x = a:1:b
# This can be any sort of array data, as long as
# length(x) == length(y)
y = y[1:100] # Function application by broadcasting
# Interpolations
#itp_linear = linear_interpolation(x, y)
itp_cubic = cubic_spline_interpolation(x, y)
# Interpolation functions
#f_linear(x) = itp_linear(x)
f_cubic(x) = itp_cubic(x)
# Plots
width, height = 1500, 800 # not strictly necessary
x_new = a:1:b # smoother interval, necessary for cubic spline

scatter(x, y, markersize=10,label="Data points")
#plot!(f_linear, x_new, w=3,label="Linear interpolation")
plot!(f_cubic, x_new, linestyle=:dash, w=3, label="Cubic Spline interpolation")
plot!(size = (width, height))
plot!(legend = :bottomleft)

In [ ]:
for i in 1:length(y)
    print(y[i],",")
end

### Read Wind site details

In [ ]:
#####################
using CSV
using Dates, DataFrames
using NativeFileDialog
using Plots, Printf

##################################################################################################
##################################################################################################
##################################################################################################

# Widen screen for better viewing
##display("text/html", "<style>.container { width:100% !important; }</style>")
display(HTML("<style>.jp-Cell { width: 120% !important; }</style>"))

# select directory
wind_directory = ".\\Wind_Data\\"

# build list of all csv files in selected directory
wind_files = filter(x->occursin("weatherdata",x), readdir(wind_directory));

# remove the "." files
wind_files = wind_files[first.(wind_files,1) .!= "."]

#try

    # select the wind data filfie
    wind_data_file = pick_file(pwd() * "\\Wind_Data\\")

    # create a df to hold the wind data
    wind_data_df = CSV.read(wind_data_file, DataFrame);

##    wind_data_df.Date += Hour(10)    # Checked with BoM on 21/05/2024, advised that wind data is in Local Time (NOT UTC!)
#end

# get site name
wind_site_name = split(uppercase.(rsplit(wind_data_file,"\\"))[end],"_")[1]

# Read BoM weather station details (see http://www.bom.gov.au/qld/observations/map.shtml)
weather_station_df = CSV.read(".\\weather_stations.txt", DataFrame)

wind_latitude = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Latitude[1]
wind_longitude = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Longitude[1]
wind_height = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Height[1]

z₀ = 0.002    # roughness length
z = wind_height
U₁₀ = wind_data_df.Wind_Speed .* log(10/z₀) ./ log(z/z₀)

p1 = Plots.plot(wind_data_df.Date, U₁₀, lc=:red, lw=:2, ls=:dot, ylims=(0, maximum(wind_val)*1.1),
        label=wind_site_name*" weather station U10", yaxis="Wind Speed (km/hr)")
p1 = Plots.plot!(date_val, wind_val, lc=:blue, lw=:2, alpha=:0.5, label="Calculated U10")

p2 = Plots.plot(wind_data_df.Date, wind_data_df.Wind_Direction, lc=:red, lw=:2, ls=:dot, ylims=(0, 360), yflip=:true,
        label=wind_site_name*" weather station Mean Direction", yaxis="Wind Direction (ᵒ)")
p2 = Plots.plot!(date_val, dir_val, lc=:blue, lw=:2, alpha=:0.5, label="Calculated Mean Direction")
##==
tm_tick = range(date_val[1],date_val[end],step=Day(1))
ticks = Dates.format.(tm_tick,"mm-dd")
#==
tm_tick = range(date_val[1],date_val[end],step=Hour(6))
ticks = Dates.format.(tm_tick,"dd HH:MM")
==#
title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
plot_file = ".\\Plots\\"*title*"_wind_plot.png"

p1_plot = Plots.plot(p1, p2, layout=(2,1), xlims=(date_val[1],date_val[end]), xticks=(tm_tick,ticks), xtickfontsize=7, xaxis="Date", 
    ytickfontsize=8, suptitle=title, titlefontsize=10, framestyle = :box, guidefontsize=10, 
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,800), gridlinewidth=0.5, gridstyle=:dot)

try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_plot)

In [ ]:
using Polynomials
using Plots

function best_fit(x, y, title, label_x, limits, tick_vals)
#########################################
    
    x = Float64.(x); y = Float64.(y)
    # Fit a linear model (first-degree polynomial)
    fit_result = Polynomials.fit(x, y, 1)
    
    # Generate the best-fit line function
    best_fit_line(x) = fit_result(x)
    
    # Plot data points
    px = Plots.scatter(x, y, markersize=:2, label="", aspect_ratio=:true, title=title, 
        xaxis=wind_site_name*label_x, xticks=tick_vals, 
        yaxis="Calculated "*label_x, yticks=tick_vals,
        xlims=limits, ylims=limits,
        xrange=limits, yrange=limits)
    
    # Plot best-fit line
    px = Plots.plot!(x, best_fit_line, label="")
    
    return(px)

end    # best_fit()


observed_df = DataFrame(Date=wind_data_df.Date, Obs_U10=U₁₀, Obs_Dir=wind_data_df.Wind_Direction)
calculated_df = DataFrame(Date=date_val, Calc_U10=wind_val, Calc_Dir=dir_val)
combined_df = innerjoin(observed_df, calculated_df, on = :Date, makeunique = :true)

title = split(split(hxv_files[1],"\\")[end],"_")[1]*"_"*monthname(displacement_df.Date[1])*"_"*string(year(displacement_df.Date[1]))
plot_file = ".\\Plots\\"*title*"_scatter_plot.png"

p1 = best_fit(combined_df.Obs_U10,combined_df.Calc_U10, "U10 values", " U10", (0,60), (0:10:60))
p2 = best_fit(combined_df.Obs_Dir,combined_df.Calc_Dir, "Directions", " directions", (0,360), (0:45:360))

p1_2_scatter = Plots.plot(p1, p2, layout=(1,2), 
    suptitle=title, titlefontsize=10, guidefontsize=10, 
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    bottommargin = 5Plots.mm, grid=true, size=(1400,600), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)


try
                                                
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p1_2_scatter)

### Do POLAR PLOTS

In [ ]:
using CairoMakie

function plot_polar(fig, displacement_df, row, col, total, spec_max)
####################################################################
    
    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    time_string  = Dates.format(displacement_df.Date[total], "HH:MM")

    aa = length(Chh)

    r = 1:20:aa
    ρ = r ./ (aa/nyquist) 

    θ = 0:pi/180:2pi

    mat =  []

    for j in r

        for i in θ

            push!(mat,Chh[j] * (a1[j]*cos(i) + b1[j]*sin(i) + a2[j]*cos(2i) + b2[j]*sin(2i)))

        end

    end

    mat[mat .< 0] .= 0

    mat = reshape(mat, length(θ), length(r))
    time_string = Dates.format(displacement_df.Date[total], "HH:MM")

    ax = CairoMakie.PolarAxis(fig[row, col],
    thetaticklabelsize = 15,  
    rlimits=(0,0.4), rticklabelsize=15, rticks=0:0.2:0.4, rgridwidth=0.5, rtickangle=180, rminorgridvisible=true, rminorgridstyle=:dot,
    theta_0=-pi/2, thetagridwidth=0.5, thetaminorgridvisible=true, thetaminorgridstyle=:dot, thetaminorticks=IntervalsBetween(3), 
    direction=-1, width=330, height=310, title=time_string, titlesize=18,
    )

    CairoMakie.contourf!(ax, θ, ρ, Float64.(mat), colormap=Reverse(:Spectral_11), levels = 0.2:0.1:1, mode = :relative,)
##    CairoMakie.contour!(ax, θ, ρ, Float64.(mat), cmap=Reverse(:Spectral))
    
    return(fig)
    
    end    # plot_polar()


#######################################################################################
#######################################################################################
#######################################################################################    

fig = CairoMakie.Figure(size=(1200, 2500))

date_string = split(hxv_directory,"\\")[end]
println("\nSpectral peak for "*date_string*" occurred at "*Dates.format(displacement_df.Date[argmax(maximum.(displacement_df.Chh))], "HH:MM"))
    
supertitle = fig[0, :] = CairoMakie.Label(fig, split(hxv_directory, "\\")[2]*" "*split(hxv_directory, "\\")[3],
    fontsize = 30, color = (:grey, 0.75))

# get the highest energy value for the day
spec_max = maximum(maximum.(displacement_df.Chh))

total=0
println("\nPreparing plots now:")
for row = 1:8

    for col ∈ 1:6

        total += 1        
        
        if (mod(total,10) == 0)
            print(string(total))
        else
            print(".")
        end
        
        try
            
            fig = plot_polar(fig, displacement_df, row, col, total, spec_max)
        
        catch
            
            println("Alert: Not all 48 records available for this day")
            break
            
        end    
    
    end
    
end

CairoMakie.Colorbar(fig[9, :], limits=(0, round(spec_max, digits=1, RoundUp)), label="Spectral Density (m²/Hz.)", labelsize=:20, 
        width=500, height=30, vertical=false, flipaxis=false, colormap=Reverse(:ocean))

resize_to_layout!(fig)

fig
##==
try
    site_string = ".\\Plots\\"*split(hxv_directory, "\\")[2]*"_"*split(hxv_directory, "\\")[3]
    CairoMakie.save(site_string*"_"*date_string*"_polar_plots.png", fig, px_per_unit = 1)
    println("\nPlot file saved as "*site_string*"_"*date_string*"_polar_plots.png")
catch
    "Alert: Plot not saved!"
end
    
#==#

display(fig)

### End Do Polar Plots

### Do SPECTRAL PLOTS

In [ ]:
using CairoMakie

function plot_spectra_and_direction(fig, displacement_df, row, col, total, spec_max)
####################################################################
    
    fhh = displacement_df.fhh[total]
    Chh = displacement_df.Chh[total]
    a1 = displacement_df.a1[total]
    b1 = displacement_df.b1[total] 
    a2 = displacement_df.a2[total] 
    b2 = displacement_df.b2[total]
    f2 = displacement_df.f2[total]
    Pden2 = displacement_df.Pden2[total]
    time_string  = Dates.format(displacement_df.Date[total], "HH:MM")

    # Calculate the Centred Fourier Coefficients
    θ₀ = atan.(b1,a1)
    m1 = (a1.^2 .+ b1.^2).^0.5
##    m2 = a2.*cos.(2*θ₀) .+ b2.*sin.(2*θ₀)
##    n2 = -a2.*sin.(2*θ₀) .+ b2.*cos.(2*θ₀)

    # Calculate the spread
    σc = (2 .* (1 .- m1)).^0.5;

    direction = mod.(rad2deg.(atan.(b1,a1)),360)

    ax1 = fig[row,col] = Axis(fig, limits=(0, 0.64, 0, 360),
        xlabel="Frequency (Hertz)", ylabel = "Direction (⁰)", yreversed=true, yticks=0:45:360, width=330, height=310, title=time_string, titlesize=18)
    ax1.xlabelsize=20; ax1.ylabelsize=20
        
    ax2 = fig[row,col] = Axis(fig,  limits=(0, 0.64, 0, spec_max),
        ylabel="Spectral Density (m²/Hz.)", yaxisposition=:right, yticklabelalign=(:left, :center), ylabelrotation=-pi/2)
    ax2.xlabelsize=20; ax2.ylabelsize=20
        
    hidedecorations!(ax2, ticks=false, ticklabels=false, label=false)

    CairoMakie.lines!(ax1, fhh, direction, linewidth=0.5, color=:grey)
    CairoMakie.band!(ax1, fhh, direction .+ rad2deg.(σc), direction .- rad2deg.(σc), fillrange = direction .- rad2deg.(σc), color=(:grey, 0.125), )

    CairoMakie.lines!(ax2, f2, Pden2, color=:red, linewidth=:2)
    CairoMakie.band!(ax2, f2, Pden2, 0, fillrange = direction .- rad2deg.(σc), color=(:red, 0.125))

##    CairoMakie.lines!(ax2, f2, Pden2, color=Pden2, linewidth=:1, colormap=Reverse(:ocean))
##    CairoMakie.band!(ax2, f2, 0, Pden2, color=Pden2, colormap=Reverse(:ocean))
            
    return(fig)
    
    end    # plot_spectra_and_direction()


#######################################################################################
#######################################################################################
#######################################################################################    
using LinearAlgebra    

fig = CairoMakie.Figure(size=(1200, 3000))

date_string = split(hxv_directory,"\\")[end]
println("\nSpectral peak for "*date_string*" occurred at "*Dates.format(displacement_df.Date[argmax(maximum.(displacement_df.Chh))], "HH:MM"))
   
supertitle = fig[0, :] = CairoMakie.Label(fig, split(hxv_directory, "\\")[2]*" "*split(hxv_directory, "\\")[3],
    fontsize = 30, color = (:grey, 0.75))

total=0
println("\nPreparing plots now:")
for row = 1:8

    for col in 1:6

        total += 1        
        
        if (mod(total,10) == 0)
            print(string(total))
        else
            print(".")
        end
        
        try
            
            fig = plot_spectra_and_direction(fig, displacement_df, row, col, total, spec_max)
        
        catch
            
            println("Alert: Not all 48 records available for this day")
            break
            
        end    
    
    end
    
end

#CairoMakie.Colorbar(fig[9, 1], limits=(0, spec_max), label="Spectral Density (m²/Hz.)", size=25, vertical=false, flipaxis=false, colormap=Reverse(:Spectral_11))

resize_to_layout!(fig)

fig
##==
try
    site_string = ".\\Plots\\"*split(hxv_directory, "\\")[2]*"_"*split(hxv_directory, "\\")[3]
    CairoMakie.save(site_string*"_"*date_string*"_spectral_plots.png", fig, px_per_unit = 1)
    println("\nFile saved as "*site_string*"_"*date_string*"_spectral_plots.png")
catch
    "Alert: Plot not saved!"
end
    
#==#

display(fig)

### End Do Spectral Plots

### Plot spectra and log-normal spectra (2 plots)

In [ ]:
using GLM
using LsqFit
using LinearAlgebra
using Peaks
using Plots
using Printf
using Roots
using StatsBase
using Tk


function estimate_Uz(frequency, spectra)
################################################
    
    # Constants
    αₖ = 0.012  # Charnock constant
    z = 10
    
    # Calculate friction velocity (uₛₜₐᵣ) from wave spectra
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
    
    # Estimate roughness length (z₀) using Charnock relation
##    z₀ = αₖ * uₛₜₐᵣ^2 / g
    
    # Calculate vertical profile of horizontal wind velocity (Uz)
    Uz = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]

    @printf("Uₛₜₐᵣ estimate = %5.4f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)
    
    return(Uz)
                    
end    # convert_to_probability_distribution()



######################################################################################################
######################################################################################################
######################################################################################################

##α = 0.0081

# box the compass
dir_string = ["N", "NNE", "NE", "ENE","E", "ESE","SE","SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "N"]
dir_brgs = [0, 22.5 , 45.0 , 67.5 , 90.0 , 112.5 , 135.0 , 157.5 , 180.0 , 202.5 , 225.0 , 247.5 , 270.0 , 292.5 , 315.0 , 337.5, 360]

try
    global dates_array = string.(displacement_df.Date)
catch
    global dates_array = Dates.format.(displacement_df.Date, "yyyy-mm-dd HH:MM")
end

w = Toplevel("Select Date", 235, 650)
tcl("pack", "propagate", w, false)
f = Frame(w)
pack(f, expand=true, fill="both")

f1 = Frame(f)
lb = Treeview(f1, dates_array)

scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Tk.Button(f, "Ok")
pack(b)

println("Select a time from the menu!")
flush(stdout)

bind(b, "command") do path
                    
    global sea_swell_transition = 1
    
    file_choice = get_value(lb);
    global iii = Int(findall(x -> x==file_choice[1], dates_array)[1])

    println("Processing selection now.")
    flush(stdout)
    
    global t = displacement_df.fhh[iii]
    global y = displacement_df.Chh[iii]

    dir = displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 0.6)
    global t = t[valid]
    global y = y[valid]
    global dir = dir[valid]
    global spread = spread[valid]

    # locate ALL peaks
    global peaks, peak_vals = findmaxima(y);

    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
##    println(length(merged_freqs) == 1 ? "Unimodal spectra" : "Bimodal spectra")
    flush(stdout)

    # Initial starting point
    start = 1

    p1 = Plots.plot(ylims=(0,maximum(y)*1.05), legend=:topleft, xlabel="Frequency (Hz.)", ylabel="Spectral Density (m²/Hertz)")

    # assume we are starting with an "ocean swell"
    previous_wave_type = ""
    separation_point = 0

    # repeat for each separation frequency detected
    for ii in separation_freqs

        next = findfirst(x->x==ii, t)
        previous_wave_type, separation_point, sea_swell_transition = get_wave_types(start, next, t, y, dir, p1, previous_wave_type, separation_point, sea_swell_transition)
        start = next

    end

    # Do the final wave type
    next = findfirst(x->x==t[end], t)
    previous_wave_type, separation_point, sea_swell_transition = get_wave_types(start, next, t, y, dir, p1, previous_wave_type, separation_point, sea_swell_transition)

    if separation_point != 0

        if t[separation_point] > 0.4
            println("\nSea-swell separation point beyond 0.4 Hz")
            separation_point = 0.0
            sea_swell_transition = 1
        else
            println("\nSea-swell separation point at ",t[separation_point])
            flush(stdout)
        end
        
    end

    ## plot the directions
    subplot = Plots.twinx()
    p1 = Plots.plot!(subplot, t, dir, lw=:2, ls=:dash, lc=:grey, grid=true, gridlinewidth=0.5, gridstyle=:dot, gridalpha=0.5, ylims=(0,360), yticks=(0:30:360), yflip=:true, label="Direction", legend=:topright, ylabel="Direction (ᵒ)")

    title = dates_array[iii]
    
    p1_plot = Plots.plot(p1, size=(1550,600), color=:lightgrey, fillrange=0, fillalpha=0.1, fillcolor=:lightgrey, xlim=(0,0.64), label="", framestyle = :box,fg_legend=:transparent, bg_legend=:transparent,
                leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, bottommargin = 15Plots.mm, title=title)

    try

        plot_file = split(split(hxv_files[iii],"\\")[end],".")[1]*"_sea_swell_separation.png"
##        savefig(plot_file)
        println("\nPlot file saved as "*plot_file)
        flush(stdout)

    catch

        "Alert: Plot not saved!"

    end

    display(p1_plot)
                                
##################################################################################################################
# Now do a semi-log plot of the spectra with f-4 and f-5 curves over the wind-sea part                               
        
    # get spectral peak of wind sea
    global wind_sea_peak = sea_swell_transition + argmax(y[sea_swell_transition:end]) - 1

    if separation_point > 0.0

        # Probably OK to treat record as having some wind sea
        
        start_equilibrium = search_nearest_index(t, t[wind_sea_peak] * 1.3)
        start_saturation = search_nearest_index(t, t[wind_sea_peak] * 3)

    else

        # Probably better to treat record as ocean swell        
        swell_peak = argmax(y)
        start_equilibrium = search_nearest_index(t, t[swell_peak] * 2)
        start_saturation = length(t)

    end
                                   
    global frequency = t[sea_swell_transition:end] 
    global spectra = y[sea_swell_transition:end]
    global dirn = dir[sea_swell_transition:end] 
    global spread = spread[sea_swell_transition:end]
    
    PM, JONSWAP = calc_PM_JONSWAP(frequency, spectra)

    # Model function for f^{-4} curve
    model_f4(x, p) = p[1] .* x .^ (-4)

    # Model function for f^{-5} curve
    model_f5(x, p) = p[1] .* x .^ (-5)

    # Plot the original data
    # Generate points for the fitted f^-x curve
#    freq_fit = frequency #range(minimum(frequency), stop=maximum(frequency), length=100)

##println(sea_swell_transition)                                    
    if sea_swell_transition != 1  

        Uz_estimate = mean(estimate_Uz(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation]))
        @printf("Uz estimate = %5.2f m/s (%5.2fkm/hr)\n",Uz_estimate, Uz_estimate*3.6)
        
        p1 = Plots.plot(t, y, lw=:3, lc=:red, label="Swell\n")
        p1 = Plots.plot!(frequency, spectra, lw=:3, lc=:blue, label="Wind sea\n")
        
        p1 = Plots.vline!([t[sea_swell_transition]], lc=:orange, ls=:dashdot, label="Sea-swell transition\n")
        p1 = Plots.vline!([t[wind_sea_peak]], lc=:green, ls=:dashdot, label="Wind sea peak\n")
        p1 = Plots.vline!([t[start_equilibrium]], lc=:blue, ls=:dashdot, label="Start of equilibrium range\n")
        
        # Plot the fitted f^{-4} curve
        f4_curve = calc_model(model_f4, t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])
        p1 = Plots.plot!(t[start_equilibrium:start_saturation], f4_curve, lc=:blue, lw=:2, linestyle=:dash, label="Fit (f⁻⁴)\n")

        # Plot the fitted f^{-5} curve
        if (t[start_saturation[1]] < t[end])
            f5_curve = calc_model(model_f5, t[start_saturation:end], y[start_saturation:end])
            p1 = Plots.vline!([t[start_saturation]], lc=:red, ls=:dashdot, label="Start of Saturation range\n")
            p1 = Plots.plot!(t[start_saturation:end], f5_curve, lc=:red, lw=:2, linestyle=:dashdot, label="Fit (f⁻⁵)")
        end
        
    else
        
        if previous_wave_type == "Ocean Swell"

            # Use Mudd et al method to resolve Uz from swell
            swell_peak = argmax(spectra)
            start_equilibrium = search_nearest_index(t, t[swell_peak] * 2)
            Uz_estimate = mean(estimate_Uz(t[start_equilibrium:end], y[start_equilibrium:end]))
            @printf("Uz estimate = %5.2f m/s (%5.2fkm/hr)\n",Uz_estimate, Uz_estimate*3.6)
        
            p1 = Plots.plot(frequency, spectra, lw=:3, lc=:red, label="Ocean Swell")
            # Plot the fitted f^{-4} curve
            f4_curve = calc_model(model_f4, t[start_equilibrium:end], y[start_equilibrium:end])
            p1 = Plots.plot!(t[start_equilibrium:end], f4_curve, lc=:blue, lw=:2, linestyle=:dash, label="Fit (f⁻⁴)\n")

            # Create a DataFrame from the subset of frequencies and spectra
            data = DataFrame(ff=t[start_equilibrium:end], ss=y[start_equilibrium:end] .* t[start_equilibrium:end] .^ 4)
            # Perform linear regression to calculate the slope
            slope_model = lm(@formula(ss ~ ff), data)
            slope = coef(slope_model)[2]
            @printf("Slope of the spectra curve in the equilibrium range: %2.5f\n",slope)
        
            # Calculate Root mean square logarithmic error
            rmsle_value = rmsle(f4_curve[1:length(t[start_equilibrium:end])], y[start_equilibrium:end])
            @printf("Root mean square logarithmic error: %2.6f\n",rmsle_value)
            
        else
            
            Uz_estimate = mean(estimate_Uz(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation]))
            @printf("Uz estimate = %5.2f m/s (%5.2fkm/hr)\n",Uz_estimate, Uz_estimate*3.6)

            p1 = Plots.plot(frequency, spectra, lw=:3, lc=:blue, label="Wind Sea")
            p1 = Plots.vline!([t[wind_sea_peak]], lc=:green, ls=:dashdot, label="Wind sea peak\n")
            p1 = Plots.vline!([t[start_equilibrium]], lc=:blue, ls=:dashdot, label="Start of equilibrium range\n")
            
            # Plot the fitted f^{-4} curve
            f4_curve = calc_model(model_f4, t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])
            p1 = Plots.plot!(t[start_equilibrium:start_saturation], f4_curve, lc=:blue, lw=:2, linestyle=:dash, label="Fit (f⁻⁴)\n")

            # Plot the fitted f^{-5} curve
            if (t[start_saturation[1]] < t[end])
                f5_curve = calc_model(model_f5, t[start_saturation:end], y[start_saturation:end])
                p1 = Plots.vline!([t[start_saturation]], lc=:red, ls=:dashdot, label="Start of Saturation range\n")
                p1 = Plots.plot!(t[start_saturation:end], f5_curve, lc=:red, lw=:2, linestyle=:dashdot, label="Fit (f⁻⁵)")
            end
    
        end
            
    end
        
    p1 = Plots.plot!(frequency, PM, lc=:pink, lw=:2, label="PM\n")
    p1 = Plots.plot!(frequency, JONSWAP, lc=:green, lw=:2, label="JONSWAP\n")
    
    try
        title = split(split(hxv_files[1],"\\")[end],"T")[1]*" "*displacement_df.Date[iii]
    catch
        title = split(split(hxv_files[1],"\\")[end],"T")[1]*" "*Dates.format(displacement_df.Date[iii], "yyyy-mm-dd HH:MM")
    end

    # determine minimum frequency for lower x-axis plotting limit
    min_freq = (minimum(frequency) < 0.2 ? 0 : 0.2)

    plot_p1 = Plots.plot(p1, size=(1500,600), framestyle = :box,
            xlims=(0.0,0.64), xlabel="Frequency (Hz)", yaxis=:log10, ylims=(0.0001,maximum(y)*1.1),
            ylabel="Spectral Density (m²/Hz.)", yminorticks=10, grid=:true, minorgrid=:true,
            fg_legend=:transparent, bg_legend=:transparent, legend=:bottomleft,  legendfontsize=12, 
            leftmargin = 15Plots.mm,  rightmargin = 15Plots.mm,  bottommargin = 15Plots.mm, title=title)

try

        plot_file = replace(replace(replace(title*"_equilibrim.png", "-" => "_"), ":" => ""), " " => "_")
        savefig(plot_file)
        println("\nPlot file saved as "*plot_file)
        flush(stdout)

    catch

        "Alert: Plot not saved!"

    end
    
    # Show the plot
    display(plot_p1)
    
end

### End Plot spectra and log-normal spectra

In [ ]:
using GLM
using LsqFit
using LinearAlgebra
using Peaks
using Plots
using Printf
using Roots
using StatsBase
using Tk

# Calculate RMSLE - reject record if > 10^-1.6
# Refer Voermans et al (2020) p.5
function rmsle(predicted, actual)
#################################    
    n = length(predicted)
    
    # Compute the element-wise logarithm of predicted and actual values
    log_predicted = log.(predicted .+ 1)  # Adding 1 to avoid log(0) issues
    log_actual = log.(actual .+ 1)
    
    # Compute the element-wise squared differences
    squared_diff = (log_predicted .- log_actual).^2
    
    # Compute the mean of the squared differences
    mean_squared_diff = sum(squared_diff) / n
    
    # Take the square root of the mean to obtain RMSLE
    rmsle_value = √(mean_squared_diff)
    
    return (rmsle_value)
    
end    # rmsle()


function estimate_Uz(frequency, spectra)
################################################
    
    # Constants
    αₖ = 0.012  # Charnock constant
    z = 10
    
    # Calculate friction velocity (uₛₜₐᵣ) from wave spectra
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
    
    # Estimate roughness length (z₀) using Charnock relation
##    z₀ = αₖ * uₛₜₐᵣ^2 / g
    
    # Calculate vertical profile of horizontal wind velocity (Uz)
    Uz = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]

    @printf("Uₛₜₐᵣ estimate = %5.4f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)
    
    return(Uz)
                    
end    # convert_to_probability_distribution()



# box the compass
dir_string = ["N", "NNE", "NE", "ENE","E", "ESE","SE","SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "N"]
dir_brgs = [0, 22.5 , 45.0 , 67.5 , 90.0 , 112.5 , 135.0 , 157.5 , 180.0 , 202.5 , 225.0 , 247.5 , 270.0 , 292.5 , 315.0 , 337.5, 360]

p1 = Plots.plot(ylims=(0,maximum(10)*1.05), legend=:topleft, xlabel="Frequency (Hz.)", ylabel="Spectral Density (m²/Hertz)")

date_val = []
wind_val = []

for iii = 1: nrow(displacement_df)
    
    global sea_swell_transition = 1
    
    global date = Dates.format( displacement_df.Date[iii], "yyyy-mm-dd HH:MM")
    global t = displacement_df.fhh[iii]
    global y = displacement_df.Chh[iii]

    dir = displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 0.6)
    global t = t[valid]
    global y = y[valid]
    global dir = dir[valid]
    global spread = spread[valid]

    # locate ALL peaks
    global peaks, peak_vals = findmaxima(y);

    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
##    println(length(merged_freqs) == 1 ? "Unimodal spectra" : "Bimodal spectra")
    flush(stdout)

    # Initial starting point
    start = 1

    # assume we are starting with an "ocean swell"
    previous_wave_type = ""
    separation_point = 0

    # repeat for each separation frequency detected
    for ii in separation_freqs

        next = findfirst(x->x==ii, t)
        previous_wave_type, separation_point, sea_swell_transition = get_wave_types(start, next, t, y, dir, p1, previous_wave_type, separation_point, sea_swell_transition)
        start = next

    end

    # Do the final wave type
    next = findfirst(x->x==t[end], t)
    previous_wave_type, separation_point, sea_swell_transition = get_wave_types(start, next, t, y, dir, p1, previous_wave_type, separation_point, sea_swell_transition)

    if separation_point != 0

        if t[separation_point] > 0.4
##            println("\nSea-swell separation point beyond 0.4 Hz")
            separation_point = 0.0
            sea_swell_transition = 1
        else
##            println("\nSea-swell separation point at ",t[separation_point])
            flush(stdout)
        end
        
    end

##################################################################################################################
    
    # get spectral peak of wind sea
    global wind_sea_peak = sea_swell_transition + argmax(y[sea_swell_transition:end]) - 1

    if separation_point > 0.0

        # Probably OK to treat record as having some wind sea
        
        start_equilibrium = search_nearest_index(t, t[wind_sea_peak] * 1.3)
        start_saturation = search_nearest_index(t, t[wind_sea_peak] * 3)

    else

        # Probably better to treat record as ocean swell        
        swell_peak = argmax(y)
        start_equilibrium = search_nearest_index(t, t[swell_peak] * 2)
        start_saturation = length(t)

    end
                                   
    global frequency = t[sea_swell_transition:end] 
    global spectra = y[sea_swell_transition:end]
    global dirn = dir[sea_swell_transition:end] 
    global spread = spread[sea_swell_transition:end]
    
    PM, JONSWAP = calc_PM_JONSWAP(frequency, spectra)

    # Model function for f^{-4} curve
    model_f4(x, p) = p[1] .* x .^ (-4)

    # Model function for f^{-5} curve
    model_f5(x, p) = p[1] .* x .^ (-5)

    # Plot the original data
    # Generate points for the fitted f^-x curve
#    freq_fit = frequency #range(minimum(frequency), stop=maximum(frequency), length=100)

##println(sea_swell_transition)                                    
    if sea_swell_transition != 1  

        Uz_estimate = mean(estimate_Uz(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation]))
        @printf("%s Uz estimate = %5.2f m/s (%5.2fkm/hr)\n", date, Uz_estimate, Uz_estimate*3.6)
        
        
        # Plot the fitted f^{-4} curve
        f4_curve = calc_model(model_f4, t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])

        # Plot the fitted f^{-5} curve
        if (t[start_saturation[1]] < t[end])
            f5_curve = calc_model(model_f5, t[start_saturation:end], y[start_saturation:end])
        end
        
    else
        
        if previous_wave_type == "Ocean Swell"

            # Use Mudd et al method to resolve Uz from swell
            swell_peak = argmax(spectra)
            start_equilibrium = search_nearest_index(t, t[swell_peak] * 2)
            Uz_estimate = mean(estimate_Uz(t[start_equilibrium:end], y[start_equilibrium:end]))
            @printf("%s Uz estimate = %5.2f m/s (%5.2fkm/hr)\n", date, Uz_estimate, Uz_estimate*3.6)
        
            # Plot the fitted f^{-4} curve
            f4_curve = calc_model(model_f4, t[start_equilibrium:end], y[start_equilibrium:end])

            # Create a DataFrame from the subset of frequencies and spectra
            data = DataFrame(ff=t[start_equilibrium:end], ss=y[start_equilibrium:end] .* t[start_equilibrium:end] .^ 4)
            # Perform linear regression to calculate the slope
            slope_model = lm(@formula(ss ~ ff), data)
            slope = coef(slope_model)[2]
##            @printf("Slope of the spectra curve in the equilibrium range: %2.5f\n",slope)
        
            # Calculate Root mean square logarithmic error
            rmsle_value = rmsle(f4_curve[1:length(t[start_equilibrium:end])], y[start_equilibrium:end])
##            @printf("Root mean square logarithmic error: %2.6f\n",rmsle_value)
            
        else
            
            Uz_estimate = mean(estimate_Uz(t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation]))
            @printf("%s Uz estimate = %5.2f m/s (%5.2fkm/hr)\n", date, Uz_estimate, Uz_estimate*3.6)

            # Plot the fitted f^{-4} curve
            f4_curve = calc_model(model_f4, t[start_equilibrium:start_saturation], y[start_equilibrium:start_saturation])

            # Plot the fitted f^{-5} curve
            if (t[start_saturation[1]] < t[end])
                f5_curve = calc_model(model_f5, t[start_saturation:end], y[start_saturation:end])
            end
    
        end
            
    end

    push!(date_val, displacement_df.Date[iii])
    push!(wind_val, Uz_estimate)

end

println("Done!")

### Plot weather station wind v's calculated wind

In [ ]:
U₁₀ = wind_data_df.Wind_Speed .* log(10/z₀) ./ log(19.8/z₀)

p1 = Plots.plot(wind_data_df.Date, U₁₀, lc=:red, lw=:2, ls=:dot, label="Weather station U10")
p1 = Plots.plot!(date_val, wind_val.*3.6, lc=:blue, lw=:2, alpha=:0.5, label="Calculated U10")

if Dates.Period(date_val[end] - date_val[1]) > Dates.Day(1)
    tm_tick = range(date_val[1],date_val[end],step=Day(1))
    ticks = Dates.format.(tm_tick,"dd")
else
    tm_tick = range(date_val[1],date_val[end],step=Hour(1))
    ticks = Dates.format.(tm_tick,"HH")
end

title = split(split(hxv_files[1],"\\")[end],"_")[1] * "_" * Dates.monthname(displacement_df.Date[1]) * "_" * string(Dates.year(displacement_df.Date[1]))

p1_plot = Plots.plot(p1, xlims=(date_val[1],date_val[end]), xticks=(tm_tick,ticks), xtickfontsize=7, xaxis="Day", 
    ytickfontsize=8, yrange=(0,50), yaxis="Wind Speed (km/hr)",
    title=title, titlefontsize=10, framestyle = :box, guidefontsize=10, 
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,800), layout=(4,1), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

try

    plot_file = title*"_wind_speed.png"
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)
    flush(stdout)

catch

    "Alert: Plot not saved!"

end

# Show the plot
display(p1_plot)



In [ ]:
using LsqFit
using Roots

# Calculate RMSLE - reject record if > 10^-1.6
# Refer Voermans et al (2020) p.5
function rmsle(predicted, actual)
#################################    
    n = length(predicted)
    
    # Compute the element-wise logarithm of predicted and actual values
    log_predicted = log.(predicted .+ 1)  # Adding 1 to avoid log(0) issues
    log_actual = log.(actual .+ 1)
    
    # Compute the element-wise squared differences
    squared_diff = (log_predicted .- log_actual).^2
    
    # Compute the mean of the squared differences
    mean_squared_diff = sum(squared_diff) / n
    
    # Take the square root of the mean to obtain RMSLE
    rmsle_value = √(mean_squared_diff)
    
    return (rmsle_value)
    
end    # rmsle()


function estimate_Uz(frequency, spectra)
################################################
    
    # Constants
    αₖ = 0.012  # Charnock constant
    z = 10
    
    # Calculate friction velocity (uₛₜₐᵣ) from wave spectra
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
    
    # Estimate roughness length (z₀) using Charnock relation
##    z₀ = αₖ * uₛₜₐᵣ^2 / g
    
    # Calculate vertical profile of horizontal wind velocity (Uz)
    Uz = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]

    @printf("Uₛₜₐᵣ estimate = %5.4f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)
    
    return(Uz)
                    
end    # convert_to_probability_distribution()


##################################################################################################
##################################################################################################
##################################################################################################

using GLM
using DataFrames

ff = frequency
ss = spectra
dd = dirn

# Find the peak value in the spectra
peak_value = maximum(ss)

# Find the index where the peak value occurs
peak_index = argmax(ss)

# Define the starting frequency for the power-law curves
starting_frequency_f4 = ff[peak_index] * 1.3
starting_index_f4 = findfirst(x -> x >= starting_frequency_f4, ff)
starting_frequency_f4 = ff[starting_index_f4] # reset starting frequency of equilibrium range to match its index

# Identify if start of Equilibrium range is beyond valid limit of spectra
if starting_frequency_f4 > 0.6
    
    println("Alert: Unable to process this record - Start of Equilibrium range is past valid limit of data!")
    
else

    starting_frequency_f5 = min(1.0, ff[peak_index] * 3.0)
    starting_index_f5 = findfirst(x -> x >= starting_frequency_f5, ff)
    
    if (starting_frequency_f5 > 0.6)
        println("Start of saturation range is beyond Nyquist frequency!")
        starting_index_f5 = length(ff)
    end

    starting_frequency_f5 = ff[starting_index_f5] # reset starting frequency of saturation range to match its index

    # Select the frequencies and spectra for the power-law curves
    ff_f4 = ff[starting_index_f4:end]
    ss_f4 = ss[starting_index_f4:end]

    ff_f5 = ff[starting_index_f5:end]
    ss_f5 = ss[starting_index_f5:end]

    # Define the model function: A * f^n + offset
    model_f4(x, p) = p[1] .* x .^ (-4) .+ p[2]
    model_f5(x, p) = p[1] .* x .^ (-5) .+ p[2]

    # Perform the curve fitting for both power-law curves
    fit_result_f4 = curve_fit(model_f4, ff_f4, ss_f4, [1.0, 1.0])
    fit_result_f5 = curve_fit(model_f5, ff_f5, ss_f5, [1.0, 1.0])

    # Extract the fitted parameters for both power-law curves
    amplitude_f4 = fit_result_f4.param[1]
    offset_f4 = fit_result_f4.param[2]

    amplitude_f5 = fit_result_f5.param[1]
    offset_f5 = fit_result_f5.param[2]

    f4_spectra = model_f4(ff_f4, fit_result_f4.param)
    f5_spectra = model_f5(ff_f5, fit_result_f5.param)

    PM, JONSWAP = calc_PM_JONSWAP(ff, ss)

    # Define the subset of frequencies and spectra for the range of interest
    ff_subset = ff[starting_index_f4:starting_index_f5]
    ss_subset = ss[starting_index_f4:starting_index_f5] .* ff_subset.^4

    # Create a DataFrame from the subset of frequencies and spectra
    data = DataFrame(ff=ff_subset, ss=ss_subset .* ff_subset .^ 4)

    Uz_estimate = mean(estimate_Uz(ff_subset, ss_subset))

    @printf("Uz estimate = %5.2f m/s (%5.2fkm/hr)\n",Uz_estimate, Uz_estimate*3.6)

    # Perform linear regression to calculate the slope
    slope_model = lm(@formula(ss ~ ff), data)
    slope = coef(slope_model)[2]
    @printf("Slope of the spectra curve in the equilibrium range: %2.5f\n",slope)

    # Calculate Root mean square logarithmic error
    rmsle_value = rmsle(f4_spectra[1:length(ff_subset)], ss[starting_index_f4:starting_index_f5])
    @printf("Root mean square logarithmic error: %2.6f\n",rmsle_value)

    # Calculate the intersection point
    initial_guess = starting_frequency_f5

    ##println(sea_swell_transition)

    # Plot the original data and the fitted power-law curves with offset
    p1 = Plots.plot(t,y,lw=:2, lc=:red, label="Ocean swell", ylims=(minimum(y), maximum(y)))
    p1 = Plots.plot!(ff, ss, label="Wind sea", lw=:2, lc=:blue)
    p1 = Plots.plot!(ff_f4, f4_spectra, label="f⁻⁴", lw=:1, lc=:blue, ls=:dash)

    if starting_frequency_f5 < 0.6
        p1 = Plots.plot!(ff_f5, f5_spectra, label="f⁻⁵", lw=:1, lc=:red, ls=:dashdot)
    end

    p1 = Plots.plot!(ff, PM, lc=:pink, lw=:2, label="PM\n")
    p1 = Plots.plot!(ff, JONSWAP, lc=:green, lw=:2, label="JONSWAP\n")

    p1 = Plots.vline!([ff[peak_index]], lw=:1, lc=:red, ls=:dot, label="Peak frequency")
    p1 = Plots.vline!([starting_frequency_f4], lw=:2, lc=:lightblue, ls=:dash, label="Start of Equilibrium range")

    if starting_frequency_f5 < 0.6
        p1 = Plots.vline!([starting_frequency_f5], lw=:2, lc=:pink, ls=:dashdot, label="Start of Saturation range")
    end

    p1 = Plots.vspan!([starting_frequency_f4,starting_frequency_f5], color=:lightgrey, alpha=:0.125, label="")

    try
        intersection_point = find_zero(x -> model_f4(x, fit_result_f4.param) - model_f5(x, fit_result_f5.param), initial_guess)

        if intersection_point < 0.6
            p1 = Plots.vline!([intersection_point], lc=:grey, ls=:dash, label="Intersection of f⁻⁴ and f⁻⁵")
        end

    catch
        println("Alert: f⁻⁴ and f⁻⁵ curves don't intersect!")
    end

    title=split(split(hxv_files[1],"\\")[end],"T")[1]*" "*dates_array[iii]

    # Show the plot
    plot_p1 = Plots.plot(p1, size=(1500,600), framestyle = :box,
        xlims=(minimum(t),0.64), xlabel="Frequency (Hz)", xticks = 0.0:0.1:0.64,
        ylims=(minimum(y),maximum(y)*1.1), ylabel="Spectral Density (m²/Hz.)", yminorticks=10, yaxis=:log10, 
        grid=:true, minorgrid=:true,
        fg_legend=:transparent, bg_legend=:transparent, legendfontsize=10, #legend=:bottomleft,  
        leftmargin = 15Plots.mm,  rightmargin = 15Plots.mm,  bottommargin = 15Plots.mm, title=title)

    display(plot_p1)
    
end

In [ ]:
function estimate_Uz(frequency, spectra)
################################################
    
    # Constants
    αₖ = 0.012  # Charnock constant
    z = 10
    
    # Calculate friction velocity (uₛₜₐᵣ) from wave spectra
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
    
    # Estimate roughness length (z₀) using Charnock relation
##    z₀ = αₖ * uₛₜₐᵣ^2 / g
    
    # Calculate vertical profile of horizontal wind velocity (Uz)
    Uz = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]

    @printf("Uₛₜₐᵣ estimate = %5.4f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)
    
    return(Uz)
                    
end    # convert_to_probability_distribution()


##################################################################################################
##################################################################################################
##################################################################################################

ff = frequency
ss = spectra
dd = dir

# Find the peak value in the spectra
peak_value = maximum(ss)

# Find the index where the peak value occurs
peak_index = argmax(ss)

# Define the starting frequency for the power-law curves
starting_frequency_f4 = ff[peak_index] * 1.3

# Identify if start of Equilibrium range is beyond valid limid of spectra
if starting_frequency_f4 > 0.6
    
    println("Alert: Unable to process this record - Start of Equilibrium range is past valid limit of data!")
    
else
    
    frequency_subset = ff_subset
    spectra_subset = ss[starting_index_f4:starting_index_f5]
    spread_subset = spread[starting_index_f4:starting_index_f5]
    direction_subset = dd[starting_index_f4:starting_index_f5]

    Uz_estimate = mean(estimate_Uz(frequency_subset, spectra_subset))

    @printf("Uz estimate = %5.2f m/s (%5.2fkm/hr)",Uz_estimate, Uz_estimate*3.6)
    
end

### Calculate regression slope closest to zero

In [ ]:
using GLM

# Function to calculate linear regression slope
function calculate_slope(x, y)
    model = lm(@formula(y ~ x), DataFrame(x=x, y=y))
    return coef(model)[2]  # Extract the slope coefficient
end

window = 19

start_index = starting_index_f4
end_index = start_index + window

# address issue where segment is less than 20 values before reaching starting_index_f5
if end_index > starting_index_f5
    end_index = length(ff)
end

slopes = Float64[]
start_freqs_index = []

while end_index <= starting_index_f5
    
    segment = ss[start_index:end_index]
    freq_range = ff[start_index:end_index]

    # Normalize spectra by multiplying with f^4
    normalized_segment = segment .* freq_range.^4
    
    # Calculate linear regression slope for the normalized segment
    x = 1:length(normalized_segment)
    slope = coef(lm(@formula(y ~ x), DataFrame(x=x, y=normalized_segment)))[2]
    
    push!(slopes, slope)
    push!(start_freqs_index,start_index)
#    println(ff[start_index]," ",ff[end_index]," ",freq_range," ",slope)

    start_index += 1
    end_index += 1

end

# Find the slope closest to 0
closest_slope_index = argmin(abs.(slopes))
closest_slope = slopes[closest_slope_index]

closest_start_freq = ff[start_freqs_index[closest_slope_index]];

println("Done!")

### Explore Equilibrium frequency range fitting method - Mudd et al (2024) Figure 2(b) p.3

In [ ]:
p1 = Plots.plot(ff,ss.*ff.^4, lc=:blue, lw=:2, label="E(f)f⁴")

p1 = Plots.plot!(ff[starting_index_f4:starting_index_f5], ss[starting_index_f4:starting_index_f5].*ff[starting_index_f4:starting_index_f5].^4, lc=:yellow, lw=:4, yaxis=:log10, label="")
p1 = Plots.plot!(ff[starting_index_f4:starting_index_f5], ss[starting_index_f4:starting_index_f5].*ff[starting_index_f4:starting_index_f5].^4, lc=:red, lw=:2, yaxis=:log10, label="")

p1 = Plots.vline!([starting_frequency_f4], lw=:2, lc=:lightblue, ls=:dash, label="Start of Equilibrium range")
p1 = Plots.vline!([starting_frequency_f5], lw=:2, lc=:pink, ls=:dashdot, label="Start of Saturation range")

p1 = Plots.vspan!([starting_frequency_f4,starting_frequency_f5], color=:lightgrey, alpha=:0.125, label="")

f4_equi_fit = mean(ss[starting_index_f4:starting_index_f5].*ff[starting_index_f4:starting_index_f5].^4)

#p1 = Plots.hline!([f4_equi_fit], label="Avg. Equilibrium range fit")
if end_index < starting_index_f5
    p1 = Plots.plot!([ff[starting_index_f4], ff[starting_index_f5]],[f4_equi_fit, f4_equi_fit], lw=2, ls=:dash, label="Avg. Equilibrium range fit")
end

bestfit = starting_index_f4 + closest_slope_index - 1

if bestfit+window > starting_index_f5
    window = starting_index_f5 - starting_index_f4
end

best_fit_ss = mean(ss[bestfit:bestfit+window].*ff[bestfit:bestfit+window].^4)

p1 = Plots.plot!([ff[bestfit], ff[bestfit+window]],[best_fit_ss, best_fit_ss], lc=:red, lw=:3, label="Best-fit value from slope-averaging")

#####################################################################
# Constants
αₖ = 0.012  # Charnock constant
z = 10

# Calculate uₛₜₐᵣ value
uₛₜₐᵣ = best_fit_ss * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
# Estimate roughness length (z₀) using Charnock relation
##z₀ = αₖ * uₛₜₐᵣ^2 / g

@printf("Uₛₜₐᵣ estimate = %5.2f m/s\n",uₛₜₐᵣ)
@printf("z₀ estimate = %5.4f m\n",z₀)

# Calculate vertical profile of horizontal wind velocity (Uz)
U₁₀ = mean([uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)])
@printf("U₁₀ estimate = %5.2f m/s (%5.2f km/hr)",U₁₀, U₁₀*3.6)
#####################################################################

plot_p1 = Plots.plot(p1, size=(1500,600), framestyle = :box,
    xlims=(minimum(t),0.64), xlabel="Frequency (Hz)", xticks = 0.0:0.1:0.64,
    ylabel="Spectral Density (m²/Hz.)", yminorticks=10, yaxis=:log10, 
    grid=:true, minorgrid=:true,
    fg_legend=:transparent, bg_legend=:transparent, legendfontsize=12, #legend=:bottomleft,  
    leftmargin = 15Plots.mm,  rightmargin = 15Plots.mm,  bottommargin = 15Plots.mm, title=title)

display(plot_p1)

### Plot best-fit value from slope-averaging

In [ ]:
p1 = Plots.plot(ff,ss, lc=:blue, lw=:2, label="Wind_sea spectra")
p1 = Plots.plot!(ff[bestfit:bestfit+window],ss[bestfit:bestfit+window], lc=:yellow, lw=:4, label="")
p1 = Plots.plot!(ff[bestfit:bestfit+window],ss[bestfit:bestfit+window], lc=:red, lw=:2, label="Equilibrium range fitting")
p1 = Plots.plot!(ff_f4, f4_spectra, label="f⁻⁴", lw=:1, lc=:blue, ls=:dash)
p1 = Plots.vline!([starting_frequency_f4], lw=:2, lc=:lightblue, ls=:dash, label="Start of Equilibrium range")
p1 = Plots.vline!([starting_frequency_f5], lw=:2, lc=:pink, ls=:dashdot, label="Start of Saturation range")

p1 = Plots.vspan!([starting_frequency_f4,starting_frequency_f5], color=:lightgrey, alpha=:0.125, label="")

plot_p1 = Plots.plot(p1, size=(1500,600), framestyle = :box,
    xlims=(minimum(t),0.64), xlabel="Frequency (Hz)", xticks = 0.0:0.1:0.64,
    ylabel="Spectral Density (m²/Hz.)", yminorticks=10, 
    grid=:true, minorgrid=:true,
    fg_legend=:transparent, bg_legend=:transparent, legendfontsize=12, #legend=:bottomleft,  
    leftmargin = 15Plots.mm,  rightmargin = 15Plots.mm,  bottommargin = 15Plots.mm, title=title)

display(plot_p1)

In [ ]:
# Function to calculate wind friction velocity
function calculate_wind_friction_velocity(spectra, frequency)
    
    # Mean of spectra times frequency^4
    mean_spectra_freq4 = mean(spectra .* frequency.^4)
    
    # Calculate wind friction velocity
    uₛₜₐᵣ = (mean_spectra_freq4 * (2π)^3) / (4 * β * Ip * g)
    
    return uₛₜₐᵣ
end

# Function to calculate U₁₀ wind speed
function calculate_U₁₀_wind_speed(uₛₜₐᵣ)
    αₖ = 0.012  # Charnock constant
    
    # Calculate roughness length
##    z₀ = αₖ * uₛₜₐᵣ^2 / g

    @printf("Uₛₜₐᵣ estimate = %5.2f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)

    # Calculate U₁₀ wind speed
    U₁₀ = uₛₜₐᵣ / κ * log(10 / z₀)
    
    return U₁₀
end

# Example spectra and frequency arrays (replace with actual data)
spectra = ss[bestfit:bestfit+window]
frequency = ff[bestfit:bestfit+window]

# Calculate wind friction velocity
uₛₜₐᵣ = calculate_wind_friction_velocity(spectra, frequency)

println("Wind friction velocity: ", uₛₜₐᵣ)


# Calculate U₁₀ wind speed
U₁₀ = calculate_U₁₀_wind_speed(uₛₜₐᵣ)
@printf("U₁₀ estimate = %5.2f m/s (%5.2f km/hr)",U₁₀, U₁₀*3.6)


### Do plot of directions weighted by spectra

In [ ]:
using StatsBase

# Define a function that checks for NaN or Nothing
isnothingornan(x) = x === nothing || (isa(x, Number) && isnan(x))

# get mean direction (weighted by spectral energy values)    
dir_vals = dd[starting_index_f4:starting_index_f5]
peak = argmax(dir_vals)

weight_vals = ss[starting_index_f4:starting_index_f5]

# need to test for NaN's or Nothing in the data and remove them before tryinh to find weighted mean
index = findall(isnothingornan, dir_vals)
if !isempty(index) 
    dir_vals = deleteat!(dir_vals, index)
    weight_vals = deleteat!(weight_vals[:,1], index)
end
    
mean_direction = mean(dir_vals, Weights(weight_vals))
@printf("Mean wave direction = %5.2fᵒ",mean_direction)

Plots.plot(ff,dd, lc=:blue, lw=:2, label="Wave direction")
Plots.plot!(ff[starting_index_f4:starting_index_f5],dd[starting_index_f4:starting_index_f5], lc=:yellow, lw=:4, label="")
Plots.plot!(ff[starting_index_f4:starting_index_f5],dd[starting_index_f4:starting_index_f5], lc=:red, lw=:2, label="")

p1 = Plots.vline!([starting_frequency_f4], lw=:2, lc=:lightblue, ls=:dash, label="Start of Equilibrium range")
p1 = Plots.vline!([starting_frequency_f5], lw=:2, lc=:pink, ls=:dashdot, label="Start of Saturation range")

p1 = Plots.vspan!([starting_frequency_f4,starting_frequency_f5], color=:lightgrey, alpha=:0.125, label="")

p1 = Plots.plot!([ff[starting_index_f4], ff[starting_index_f5]],[mean_direction, mean_direction], lw=2, ls=:dash, label="Weighted-mean direction")

plot_p1 = Plots.plot(p1, size=(1500,600), framestyle = :box,
    xlims=(minimum(t),0.64), xlabel="Frequency (Hz)", xticks = 0.0:0.1:0.64,
    ylabel="Direction (ᵒ)", yminorticks=5, yflip=:true,
    grid=:true, minorgrid=:true,
    fg_legend=:transparent, bg_legend=:transparent, legendfontsize=12, #legend=:bottomleft,  
    leftmargin = 15Plots.mm,  rightmargin = 15Plots.mm,  bottommargin = 15Plots.mm, title=title, legend=:bottomleft)

display(plot_p1)

### Do 4-plot of log-normal Spectra

In [ ]:
using Colors

p1 = Plots.plot(ylabel="Spectral Density (m²/Hz.)")
title = ""

times_array = displacement_df.Date
times = displacement_df.Date[1:12]

for ii in times

    val = findfirst(==(ii), times_array)

    p1 = Plots.plot!(displacement_df.fhh[val], displacement_df.Chh[val], label=ii, lw=:1.5, palette=Plots.palette(:temperaturemap, rev=:true, length(times)))

end

times = displacement_df.Date[13:24]

p2 = Plots.plot()

for ii in times

    val = findfirst(==(ii), times_array)

    p2 = Plots.plot!(displacement_df.fhh[val], displacement_df.Chh[val], label=ii, lw=:1.5, palette=Plots.palette(:temperaturemap, rev=:true, length(times)))
    
end

times = displacement_df.Date[25:36]

p3 = Plots.plot()

for ii in times

    val = findfirst(==(ii), times_array)

    p3 = Plots.plot!(displacement_df.fhh[val], displacement_df.Chh[val], label=ii, lw=:1.5, palette=Plots.palette(:temperaturemap, rev=:true, length(times)))
    
end

times = displacement_df.Date[37:48]

p4 = Plots.plot()

for ii in times

    val = findfirst(==(ii), times_array)

    p4 = Plots.plot!(displacement_df.fhh[val], displacement_df.Chh[val], label=ii, lw=:1.5, palette=Plots.palette(:temperaturemap, rev=:true, length(times)))
    
end

title=split(split(hxv_files[1],"\\")[end],"T")[1]

plot_p1 = Plots.plot(p1, p2, p3, p4, size=(1600,450), layout=(1,4), framestyle = :box,
    xlims=(0.0,0.64), xlabel="Frequency (Hz)", ylims=(minimum(minimum.(displacement_df.Chh)),maximum(maximum.(displacement_df.Chh))),
    yminorticks=5, grid=:true, minorgrid=:true, yaxis=:log10, 
    fg_legend=:transparent, bg_legend=:transparent, legendfontsize=8, guidefontsize=:10,
    leftmargin = 7Plots.mm,  rightmargin = 1Plots.mm,  bottommargin = 10Plots.mm, suptitle=title, suptitlefontsize=:6)

try

    plot_file = replace(replace(replace(title*"_daily_spectra.png", "-" => "_"), ":" => ""), " " => "_")
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)
    flush(stdout)

catch

    "Alert: Plot not saved!"

end
    

# Show the plot
display(plot_p1)

### Process all records for sea-swell separation point

In [ ]:
function get_wave_types1(t, y, previous_wave_type, separation_point)
##############################################
    
"""
    Calls:  calc_moments()
            nearest_neighbor()
    Called by: Main
"""
    peak = argmax(y)
    fₚ = t[peak]
    Tₚ = 1/fₚ    
    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t,y)
    hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc = calc_parameters(m₋₁, m₀, m₁, m₂, m₄)
    
    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t]

    # determine whether this part of wave record is sea or swell
    # Refer https://www.researchgate.net/publication/249605099_Spectral_Partitioning_and_Identification_of_Wind_Sea_and_Swell
    ratio = y[peak] / Sf[peak]
    wave_type = (ratio < 1 ? "Ocean Swell" : "Wind Sea")

    # test whether separation between sea and swell has been found
    if (previous_wave_type == "Ocean Swell") && (wave_type == "Wind Sea")
    # Separation between swell and sea has been located
        separation_point = t[1]
    end

    return(wave_type, separation_point)
        
end    # get_wave_types1()


function calc_parameters(m₋₁, m₀, m₁, m₂, m₄)
#############################################
    
    hₘ₀ = 4√m₀
    Hᵣₘₛ = √(8m₀)
    T₀₂ = √(m₀/m₂)    # mean wave period
    T₀₁ = m₀/m₁       # significant wave period
    T₋₁₀ = m₋₁/m₀     # mean energy period Tₑ
    Tc = √(m₂/m₄)

    return(hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc)

    end    # calc_parameters()


function get_hm0_tp_pkdir_meandir(t_, y_, dir, date)
##############################################
    
    peak = argmax(y_)
    fₚ = t_[peak]
    Tₚ = 1/fₚ 

    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t_,y_)
    hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc = calc_parameters(m₋₁, m₀, m₁, m₂, m₄)

    # get peak direction and mean direction (weighted by spectral energy values)
    peak_direction = dir[peak]
    weighted_mean_direction = mean(dir, Weights(y_))

    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t_]
    
    ratio = y_[peak] / Sf[peak]    # refer Portilla et al (2009) p.117 Section 2.
    wave_type = (ratio < 1 ? "Ocean Swell" : "Wind Sea")

##    @printf("%s Hₘ₀ = %5.2fm; Hᵣₘₛ = %5.2fm; T₋₁₀ = %5.2fs; T₀₁ = %5.2fs; T₀₂ = %5.2fs; Tc = %5.2fs; Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Mean Dirn = %6.2fᵒ Wave type = %s; Ratio = %2.4f\n",
##        time_string, hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc, Tₚ, peak_direction, weighted_mean_direction, wave_type, ratio)
##    @printf("%s Hₘ₀ = %5.2fm; Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Mean Dirn = %6.2fᵒ Wave type = %s; Ratio = %2.4f\n",
##        time_string, hₘ₀, Tₚ, peak_direction, weighted_mean_direction, wave_type, ratio)
    flush(stdout)

    return(hₘ₀, Tₚ, peak_direction, weighted_mean_direction, Sf, wave_type)

    end    # get_hm0_tp_pkdir_meandir()

#################################################################################################
#################################################################################################
#################################################################################################

# define a df to hold wind sea spectral values
wind_sea_df = DataFrame([[], [], [], []], ["Date", "Frequency", "Spectra", "PM"])


# define arrays to hold sea and swell data
hm0_swell_array = []; Tp_swell_array = []; pkdir_swell_array = []; mean_dir_swell_array = []
hm0_sea_array = []; Tp_sea_array = []; pkdir_sea_array = []; mean_dir_sea_array = []
hm0_all_array = []; Tp_all_array = []; pkdir_all_array = []; mean_dir_all_array = []

for iii in 1:nrow(displacement_df)
    
    t = displacement_df.fhh[iii]
    y = displacement_df.Chh[iii]

    dir = displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 0.6)
    t = t[valid]
    y = y[valid]
    dir = dir[valid]
    spread = spread[valid] 
      
    # Initialize starting variables
    start = 1
    separation_point = 0
    wave_type = "?"
    previous_wave_type = "?"
    

    # locate ALL peaks and the separation points between them
    peaks, peak_vals = findmaxima(y);
    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)

    # get the wave parameters
    hm0, Tp, pkdir, mean_dir, Sf, wave_type_all = get_hm0_tp_pkdir_meandir(t, y, dir, displacement_df.Date[iii])
    push!(hm0_all_array, hm0); push!(Tp_all_array, Tp); push!(pkdir_all_array, pkdir); push!(mean_dir_all_array, mean_dir)

    println(displacement_df.Date[iii], " ", wave_type_all)
    
    # If there are no separation points, it means we have a Unimodal wave event
    if isempty(separation_freqs)   
        
        # populate the sea and swell arrays depending on whether unimodal event is sea or swell - NaN's will go to the other array
        if wave_type_all == "Ocean Swell"

            push!(hm0_swell_array, hm0); push!(Tp_swell_array, Tp); push!(pkdir_swell_array, pkdir); push!(mean_dir_swell_array, mean_dir)
            push!(hm0_sea_array, NaN); push!(Tp_sea_array, NaN); push!(pkdir_sea_array, NaN); push!(mean_dir_sea_array, NaN)
            
        elseif wave_type_all == "Wind Sea"
                       
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [displacement_df.Date[iii], t, y, Sf])
            
        else
            
            # should not happen!
            println("Unusual Unimodal event detected")
            
        end
            
        
    else
        
        # multimodal event, so repeat for each separation frequency detected
        for ii in separation_freqs
            
            next = findfirst(x->x==ii, t)
            wave_type, separation_point = get_wave_types1(t[start:next], y[start:next], previous_wave_type, separation_point)

            # Identify if a sea-swell separation has occurred
            if ((previous_wave_type=="Ocean Swell") && (wave_type=="Wind Sea"))
                
                break    # Exit the for loop

            end

            start = next         
            previous_wave_type = wave_type
            
        end
        
        hm0_swell, Tp_swell, pkdir_swell, mean_dir_swell, Sf_swell, wave_type_swell = get_hm0_tp_pkdir_meandir(t[1:start], y[1:start], dir[1:start], displacement_df.Date[iii])
        hm0_sea, Tp_sea, pkdir_sea, mean_dir_sea, Sf_sea, wave_type_sea = get_hm0_tp_pkdir_meandir(t[start:end], y[start:end], dir[start:end], displacement_df.Date[iii])
        

        if ((wave_type_swell=="Ocean Swell") & (wave_type_sea=="Wind Sea"))
            
            push!(hm0_swell_array, hm0_swell); push!(Tp_swell_array, Tp_swell); push!(pkdir_swell_array, pkdir_swell); push!(mean_dir_swell_array, mean_dir_swell)
            push!(hm0_sea_array, hm0_sea); push!(Tp_sea_array, Tp_sea); push!(pkdir_sea_array, pkdir_sea); push!(mean_dir_sea_array, mean_dir_sea)
            
            push!(wind_sea_df, [displacement_df.Date[iii], t[start:end], y[start:end], Sf_sea])
            
        elseif ((wave_type_swell=="Ocean Swell") & (wave_type_sea=="Ocean Swell"))
            
            push!(hm0_swell_array, hm0); push!(Tp_swell_array, Tp); push!(pkdir_swell_array, pkdir); push!(mean_dir_swell_array, mean_dir)
            push!(hm0_sea_array, NaN); push!(Tp_sea_array, NaN); push!(pkdir_sea_array, NaN); push!(mean_dir_sea_array, NaN)
            
        elseif ((wave_type_swell=="Wind Sea") & (wave_type_sea=="Wind Sea"))
            
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [displacement_df.Date[iii], t[start:end], y[start:end], Sf_sea])
            
        else # must be a Wind and Swell combination - this could be an artifact of the PM spectrum
            
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [displacement_df.Date[iii], t[start:end], y[start:end], Sf_sea])
            
        end
    
    end

#    println(displacement_df.Date[iii]*" "*wave_type_swell*" "*wave_type_sea)
    
##    println("----------------------------------------------------------------------------------------------------------")

end

# Plot Hm0 and Tp for sea and swell
dates = DateTime.([(split.(el, "_")[2])[1:end-4] for el in hxv_files], "yyyy-mm-ddTHHhMMK")
# build title string showing date range
title = Dates.format(dates[1], "yyyy-mm-dd")
annotation_string = split(hxv_directory, "\\")[3]*"_"*title

p1 = Plots.plot(dates, hm0_all_array, lc=:blue, lw=:3, label="Total", ylabel="Hm0 (m)")
p1 = Plots.plot!(dates, hm0_sea_array, lc=:red, lw=:3, label="Sea")
p1 = Plots.plot!(dates, hm0_swell_array, lc=:green, lw=:3, label="Swell")

#p2 = Plots.plot(dates,Tp_all_array, lc=:blue, lw=:2, label="Total", ylabel="Tp (s)")
p2 = Plots.plot(dates, Tp_sea_array, lc=:red, lw=:3, label="Sea", ylabel="Tp (s)")
p2 = Plots.plot!(dates, Tp_swell_array, lc=:green, lw=:3, label="Swell")

p3 = Plots.plot(dates, mean_dir_all_array, lc=:blue, lw=:3, label="Total", yflip=:true, ylabel="Mean direction (ᵒ)", legend=:bottomright)
p3 = Plots.plot!(dates, mean_dir_sea_array, lc=:red, lw=:3, label="Sea")
p3 = Plots.plot!(dates, mean_dir_swell_array, lc=:green, lw=:3, label="Swell")

p4 = Plots.plot(dates, pkdir_all_array, lc=:blue, lw=:3, label="Total", yflip=:true, ylabel="Peak direction (ᵒ)", xlabel="Time AEST", legend=:bottomright)
p4 = Plots.plot!(dates, pkdir_sea_array, lc=:red, lw=:3, label="Sea")
p4 = Plots.plot!(dates, pkdir_swell_array, lc=:green, lw=:3, label="Swell")

#p1 = Plots.annotate!(dates[1], maximum(hm0_all_array), (annotation_string, :left, :blue, 10))

tm_tick = range(dates[1],dates[end],step=Hour(2))
ticks = Dates.format.(tm_tick,"HH:MM")

title=split(split(hxv_files[1],"\\")[end],"T")[1]

p1_plot = Plots.plot(p1, p2, p3, p4, xlims=(dates[1],dates[end]), xticks=(tm_tick,ticks), xtickfontsize=7,
    ytickfontsize=8, 
    title=title, titlefontsize=10, framestyle = :box, guidefontsize=10, 
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 2Plots.mm, grid=true, size=(1600,1000), layout=(4,1), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

try

        plot_file = split(split(hxv_files[1],"\\")[end],"T")[1]*"_sea_swell_split.png"
        savefig(plot_file)
        println("\nPlot file saved as "*plot_file)
        flush(stdout)

    catch

        "Alert: Plot not saved!"

    end

display(p1_plot)

### Process all records for sea-swell separation point

In [ ]:
using Peaks


# function to calc spectral moments
function calc_moments(t,y)
#########################
    
"""
    Calls: Nil
    Called by: calc_PM_JONSWAP()
"""
    ax1 = (last(t) - first(t)) / (length(t)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s_₀₁ = 0; s₀₀ = 0; s₀₁ = 0; s₀₂ = 0; s₀₃ = 0; s₀₄ = 0;
    m₋₁ = 0; m₀ = 0; m₁ = 0; m₂ = 0; m₃ = 0; m₄ = 0

    for ii in 1:length(t)

        s_₀₁ += t[ii]^-1 * y[ii]
        s₀₀ += t[ii]^0 * y[ii]
        s₀₁ += t[ii]^1 * y[ii]
        s₀₂ += t[ii]^2 * y[ii]
        s₀₃ += t[ii]^3 * y[ii]
        s₀₄ += t[ii]^4 * y[ii]

    end

    m₋₁ = 0.5*ax1*(first(t)^-1*first(y) + 2*s_₀₁ + last(t)^0*last(y))
    m₀ = 0.5*ax1*(first(t)^0*first(y) + 2*s₀₀ + last(t)^0*last(y))
    m₁ = 0.5*ax1*(first(t)^1*first(y) + 2*s₀₁ + last(t)^1*last(y))
    m₂ = 0.5*ax1*(first(t)^2*first(y) + 2*s₀₂ + last(t)^2*last(y))
    m₃ = 0.5*ax1*(first(t)^3*first(y) + 2*s₀₃ + last(t)^3*last(y))
    m₄ = 0.5*ax1*(first(t)^4*first(y) + 2*s₀₄ + last(t)^4*last(y))        

    return(m₋₁, m₀, m₁, m₂, m₄)

    end    # calc_moments()


# calculate frequency domain parameters            
function calc_parameters(t_,y_)
###############################

"""
    Calls: calc_moments
"""
            
    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t_,y_)
    hₘ₀ = 4√m₀
    Hᵣₘₛ = √(8m₀)
    T₀₂ = √(m₀/m₂)    # mean wave period
    T₀₁ = m₀/m₁       # significant wave period
    T₋₁₀ = m₋₁/m₀     # mean energy period Tₑ
    Tc = √(m₂/m₄)
            
    return(hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc)

    end    # calc_parameters()


function modified_get_wave_types(start, next, t, y, dir, peak_vals, previous_wave_type, separation_point)
##############################################

"""
    Calls: calc_parameters
"""
    t_ = t[start:next]
    y_ = y[start:next]
    dir_ = dir[start:next]

##    p1 = Plots.plot!(t_,y_, lw=:2, fillrange=0, fillalpha=0.1, 
##        label=string(@sprintf("%0.2f to %0.2f", t[start], t[next])*"Hz.\n")) #label="$(t[start]) to $(t[next])Hz.\n")
##    p1 = Plots.vline!([t[next]], label="")
    
    peak = argmax(y_)
    fₚ = t_[peak]
    Tₚ = 1/fₚ  
            
    hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc = calc_parameters(t_,y_)         
    
    # get peak direction and mean direction (weighted by spectral energy values)
    peak_direction = dir_[peak]
    weighted_mean_direction = mean(dir_, Weights(y_))

    # box the compass
    dir_string = ["N", "NNE", "NE", "ENE","E", "ESE","SE","SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW", "N"]
    dir_brgs = [0, 22.5 , 45.0 , 67.5 , 90.0 , 112.5 , 135.0 , 157.5 , 180.0 , 202.5 , 225.0 , 247.5 , 270.0 , 292.5 , 315.0 , 337.5, 360]
        
    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t_]
##    p1 = Plots.plot!(t_, Sf, lc=:grey, lw=:2, ls=:dot, label="")

    # determine whether this part of wave record is sea or swell
    # Refer https://www.researchgate.net/publication/249605099_Spectral_Partitioning_and_Identification_of_Wind_Sea_and_Swell
    ratio = y_[peak] / Sf[peak]
    wave_type = (ratio <= 1 ? "Ocean Swell" : "Wind Sea")

    # test whether separation between sea and swell has been found
    if (previous_wave_type == "Ocean Swell") && (wave_type == "Wind Sea")
    # Separation between swell and sea has been located
        separation_point = start
    end

    previous_wave_type = wave_type
       
    # get direction string
    compass = dir_string[findfirst(x->x==nearest_neighbor(peak_direction, dir_brgs), dir_brgs)]
       
##    @printf("Hₘ₀ = %5.2fm; Hᵣₘₛ = %5.2fm; T₋₁₀ = %5.2fs; T₀₁ = %5.2fs; T₀₂ = %5.2fs; Tc = %5.2fs; Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Mean Dirn = %6.2fᵒ Wave type = %s from %s\n",
##            hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc, Tₚ, peak_direction, weighted_mean_direction, wave_type, compass)
##    flush(stdout)
    
    return(previous_wave_type, separation_point)
        
end    # modified_get_wave_types()


##################################################################################################
##################################################################################################
##################################################################################################
dates = DateTime.([(split.(el, "_")[2])[1:end-4] for el in hxv_files], "yyyy-mm-ddTHHhMMK")
separation_points_date = []
separation_points_frequency = []

println("Reading 30-minute records now:\n")
for iii in 1:nrow(displacement_df)
#==    
    if (mod(iii,100) == 0)
        print(string(iii))
    else
        print(".")
    end
==#
##    println(displacement_df.Date[iii])
    t = displacement_df.fhh[iii]
    y = displacement_df.Chh[iii]

    dir =displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 1.0)  ##0.6)
    t = t[valid]
    y = y[valid]
    dir = dir[valid]
    spread = spread[valid]

    # locate ALL peaks
    peaks, peak_vals = findmaxima(y);

    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)
##    println(length(merged_freqs) == 1 ? "Unimodal spectra" : "Bimodal spectra")

    # Initial starting point
    start = 1

##    p1 = Plots.plot(ylims=(0,maximum(y)*1.05), legend=:topleft, xlabel="Frequency (Hz.)", ylabel="Spectral Density (m²/Hertz)")

    # assume we are starting with an "ocean swell"
    previous_wave_type = "Wind Sea"
    separation_point = 0

    # repeat for each separation frequency detected
    for ii in separation_freqs

        next = findfirst(x->x==ii, t)
        previous_wave_type, separation_point = modified_get_wave_types(start, next, t, y, dir, peak_vals, previous_wave_type, separation_point)
        start = next

    end

    # Do the final wave type
    next = findfirst(x->x==t[end], t)
    previous_wave_type, separation_point = modified_get_wave_types(start, next, t, y, dir, peak_vals, previous_wave_type, separation_point)

    if separation_point != 0

        push!(separation_points_date, dates[iii])
        push!(separation_points_frequency, t[separation_point])

    end
    
end

println("Done!")

### Plot spectra over selected day

In [ ]:
x = dates
y = displacement_df.fhh

z = displacement_df.Chh[1:end]
Z = hcat(z...)

max_spec = maximum(Z)
# display plots to screen
tm_tick = range(x[1],x[end],step=Hour(1))
ticks = Dates.format.(tm_tick,"HH:MM")

p1 = Plots.contourf(x, y, Z, c=cgrad(:Spectral, rev=true), clims=(0,max_spec), levels=10, fill=true)

p1 = Plots.scatter!(separation_points_date, separation_points_frequency, mc=:yellow, marker=:diamond, label="Sea-swell separation", legendfontsize=:12, legendfontcolor=:yellow)
p1 = Plots.plot!(separation_points_date, separation_points_frequency, lc=:yellow, ls=:dash, label="")

# draw grid lines on plot
for i in 0:0.1:0.64 ##0.6
    Plots.hline!(p1, [i], lw=0.5, c=:white, label="")
end

for i in x[1]:Hour(1):x[end]
    Plots.vline!(p1, [i], lw=0.5, c=:white, label="")
end

# build title string showing date range
start_date = Dates.format(x[1], "yyyy-mm-dd HH:MM")
end_date = Dates.format(x[end], "yyyy-mm-dd HH:MM")
title=split(split(hxv_files[1],"\\")[end],"T")[1]

p1_plot = Plots.plot(p1, xlabel="Date", xlims=(x[1],x[end]), xticks=(tm_tick,ticks), xtickfontsize=7,
    ylabel="Frequency (Hz)", ylim=(0.025,0.64), ytickfontsize=8, 
    title=title, titlefontsize=10, framestyle = :box, guidefontsize=10,
    colorbar_title="Spectral Density (m²/Hertz)", colorbar_titlefontsize=8,
    fg_legend=:transparent, bg_legend=:transparent, 
    leftmargin = 15Plots.mm, bottommargin = 15Plots.mm, grid=true, size=(1600,800), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, colorbar=true)

try
    plot_date = Dates.format(displacement_df.Date[iii], "yyyy_mm")
    plot_file = ".\\Plots\\"*split(f20_file_name,"{")[1]*"_monthly_spectral_plot.png"
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)
catch
    "Alert: Plot not saved!"
end

Plots.display(p1_plot)

### Process each file in the selected directory and save the results to a JSON .csv file

In [ ]:
using CSV
using Dates, DataFrames, Distributions, DSP
using FilePathsBase


function get_displacements(arry)
#####################################
    
    displacements = []

    if length(arry[1]) == 3
    
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^2 + parse(Int, SubString.(i, 2, 2), base=16)*16^1 + parse(Int, SubString.(i, 3, 3), base=16)*16^0)
        end
        
    else
        
        for i in arry
            append!(displacements,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
        end
        
    end

    displacements[findall(>=(2048), displacements)] = 2048 .- displacements[findall(>=(2048), displacements)];
    
    return(displacements./100)
    
    end     # get_displacements()


function get_HNW(infil)
#####################################
        
#    global df = DataFrame(CSV.File(infil,header=0, delim=",", types=String));
    
    df = DataFrame(CSV.File(infil,header=0, delim=";", types=String));

    # Remove rows that do not match Datawell's data format
    df_length_ok = df[Bool[length(x)==24 for x in df.Column1],:]

    df = DataFrames.select!(insertcols!(df_length_ok, [n => getindex.(split.(df_length_ok.Column1, ","), i) for (i, n) in
                            enumerate([:Data1, :Data2, :Data3, :Data4, :Data5])]..., makeunique=true), Not(:Column1))

    colnames = ["Column1", "Column2", "Column3", "Column4", "Column5"]
    rename!(df, Symbol.(colnames))
        
    # Calculate sequence numbers
    arry = SubString.(df.Column1, 3, 4)

    global sequence = []

    for i in arry
        append!(sequence,parse(Int, SubString.(i, 1, 1), base=16)*16^1 + parse(Int, SubString.(i, 2, 2), base=16)*16^0)
    end

    # Calculate heave WSEs
    arry = SubString.(df.Column3, 1, 3);
    heave = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column3, 4, ) .* SubString.(df.Column4, 1, 2)
    north = get_displacements(arry);

    # Calculate north WSEs
    arry = SubString.(df.Column4, 3, 4) .* SubString.(df.Column5, 1, 1)
    west = get_displacements(arry);

    return(heave, north, west)

end    # get_HNW()


function fix_record_length(arry)
################################

    if length(arry) > 2304
        arry = arry[1:2304]
   elseif length(arry) < 2304
        append!(arry, zeros(2304 - length(arry)))
    end
    
    return(arry)
    
end    # fix_record_length()


function calc_wse(infil, wse_df, start_date)
#####################################    
    
    heave, north, west = get_HNW(infil)

    heave = fix_record_length(heave)
    north = fix_record_length(north)
    west = fix_record_length(west)
    
    # Identify any gaps in the recorded data
    tt = [0]
    append!(tt,diff(sequence))
    tt[tt.<0] .+= 256;
    tt1 = cumsum(tt);
    
    # only interested in 2304 records
    tt1 = tt1[tt1 .< 2304]
    
    [wse_df[tt1[i]+1,2] = heave[i] for i in eachindex(tt1)];
    [wse_df[tt1[i]+1,3] = north[i] for i in eachindex(tt1)];
    [wse_df[tt1[i]+1,4] = west[i] for i in eachindex(tt1)];
    
    return(wse_df)
    
end    # calc_wse()


# Calculate Tp5 via Read method
function calc_tp5(f2,Sf)
##########################################
    Sf_max = maximum(Sf)

    numerator = 0; denominator = 0

    Sf_sum = cumsum(Sf.*Sf_max).^5

    for i in 1:length(f2)
        w = Sf[i] / Sf_max
        numerator +=  f2[i] * w^5
        denominator += w^5
    end

    Fp5 = numerator / denominator
    
    return(1/Fp5)    # calc_tp5()

    end    # calc_tp5()


# Calculate frequency-domain parameters    
function calculate_frequency_domain_parameters(f2, spectra)
##########################################
# Calls: calc_tp5()
    
    ax1 = (last(f2) - first(f2)) / (length(f2)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s00 = 0; s01 = 0; s02 = 0; s03 = 0; s04 = 0;
    m0 = 0; m1 = 0; m2 = 0; m3 = 0; m4 = 0

    for ii in 1:length(f2)

        s00 += f2[ii]^0 * spectra[ii]
        s01 += f2[ii]^1 * spectra[ii]
        s02 += f2[ii]^2 * spectra[ii]
        s03 += f2[ii]^3 * spectra[ii]
        s04 += f2[ii]^4 * spectra[ii]

    end

    m0 = 0.5*ax1*(first(f2)^0*first(spectra) + 2*s00 + last(f2)^0*last(spectra))
    m1 = 0.5*ax1*(first(f2)^1*first(spectra) + 2*s01 + last(f2)^1*last(spectra))
    m2 = 0.5*ax1*(first(f2)^2*first(spectra) + 2*s02 + last(f2)^2*last(spectra))
    m3 = 0.5*ax1*(first(f2)^3*first(spectra) + 2*s03 + last(f2)^3*last(spectra))
    m4 = 0.5*ax1*(first(f2)^4*first(spectra) + 2*s04 + last(f2)^4*last(spectra))

    ##println("m0 = ",m0," m1 = ",m1, " m2 = ",m2, " m3 = ",m2, " m4 = ",m4)

    # calc wave parameters Hm0, Hrms, T01, T02, Tc
    Hm0 = 4*√(m0)     # Tucker & Pitt p.32 (2.2-6b)
    Hrms = √(8*m0)    # Goda 2nd. Edition p.262 (9.15)
    T01 = m0/m1          # Tucker & Pitt p.41 Table 2.2 
    T02 = √(m0/m2)    # Tucker & Pitt p.40 (2.3-2)
    Tc = √(m2/m4)     # Tucker & Pitt p.41 Table 2.2 - also see Notes

    # identify spectral peak and frequency as peak
    Fp = f2[argmax(spectra)]
    Tp = 1/Fp
    fp5 = calc_tp5(f2, spectra)
    Tp5 = 1/fp5

    # calculate spectral width vide Tucker and Pitt p.85 (5.2-8)
    # Note: for JONSWAP, v = 0.39; for PM, v = 0.425
    v = (m0*m2 / m1^2 - 1)^0.5

    # calculate Skewness vide Tucker and Pitt p.109 (5.5-17)
    Skewness = (m0^2 * m3/m1^3 - 3*v^2 - 1) / v^3;
    
    return(Hm0, Hrms, T01, T02, Tc, Tp, fp5, Tp5, Skewness)
    
end    # calculate_frequency_domain_parameters()


######################################################################################################
######################################################################################################
######################################################################################################

using FFTW
using DSP

results_df = DataFrame(Date=DateTime[], Hm0=Float64[], Hrms=Float64[], T01=Float64[], T02=Float64[], 
                       Tc=Float64[], Tp=Float64[], Tp5=Float64[], Skewness=Float64[], fhh=Vector{Float64}[], 
                       Chh=Vector{Float64}[], Dirn=Vector{Float64}[], Spread=Vector{Float64}[])

total = 0

for ii ∈ 1:nrow(displacement_df)
    
    total+=1        

    if (mod(total,10) == 0)
        print(string(total))
    else
        print(".")
    end

    
    # extract the datetime from the file name
    date_str = dates[ii]
    Hm0, Hrms, T01, T02, Tc, Tp, fp5, Tp5, Skewness = calculate_frequency_domain_parameters(displacement_df.fhh[ii], displacement_df.Chh[ii])
    
##    @printf("%s; Hm0 = %5.2fm; Hrms = %5.2fm; T01 = %5.2fs; T02 = %5.2fs; Tc = %5.2fs; Tp = %5.2fs; Tp5 = %5.2fs; Skewness = %5.4f\n",
##        Dates.format(displacement_df.Date[ii], "yyyy-mm-dd HH:MM"),Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness)
    
    push!(results_df,(dates[ii], Hm0, Hrms, T01, T02, Tc, Tp, Tp5, Skewness, displacement_df.fhh[ii], displacement_df.Chh[ii], displacement_df.Direction[ii], displacement_df.Spread[ii]))

end

using JSON

# Convert arrays to JSON string before saving
df_str = copy(results_df)
for col in [:fhh, :Chh, :Dirn, :Spread]
    df_str[!, col] = [JSON.json(v) for v in df_str[!, col]]
end

# Output file name
jfile = split(split(hxv_files[1],"\\")[end],"T")[1]*"_JSON.csv"

# Save dataframe to CSV file
println("\nWriting data to "*jfile)
CSV.write(jfile, df_str)

println("Done!")

### Read saved .CSV file and convert JSON strings back into arrays

In [ ]:
using JSON
using CSV
using Dates, DataFrames, Distributions, DSP
using FilePathsBase

##using Gtk
##using LaTeXStrings
using NativeFileDialog
using Plots
using Printf
using Statistics #, StatsPlots
##using Tk


##csv_file = "glad_2023-11-10_JSON.csv"
##csv_file = "glad_2024-01-02_JSON.csv"
##csv_file = "glad_2024-01-05_JSON.csv"
##csv_file = "glad_2024-01-12_JSON.csv"
##csv_file = "glad_2024-01-13_JSON.csv"
csv_file = "glad_2024-01-16_JSON.csv"

##csv_file = "calo_2024-05-01_JSON.csv"
##csv_file = "calo_2023-06-06_JSON.csv"
##csv_file = "calo_2023-12-23_JSON.csv"

##csv_file = "alby_2022-06-21_JSON.csv"
##csv_file = "alby_2023-01-27_JSON.csv"

##csv_file = "mool_2023-11-14_JSON.csv"

# Read CSV file to a DataFrame
println("Reading: ",csv_file); flush(stdout)
wave_data_df_str = CSV.read(csv_file, DataFrame)

# Convert JSON strings back to arrays
wave_data_df = copy(wave_data_df_str)
for col in [:fhh, :Chh, :Dirn, :Spread]
    wave_data_df[!, col] = [JSON.parse(v) for v in wave_data_df[!, col]]
end

wave_site_name = uppercase(split(csv_file, "_")[1])

println("Done!")

### Read Wind data from .csv file

In [ ]:
#####################
using CSV
using Dates, DataFrames
using NativeFileDialog
using Plots, Printf

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")
##display(HTML("<style>.jp-Cell { width: 150% !important; }</style>"))

# select directory
wind_directory = ".\\Wind_Data\\"

# build list of all csv files in selected directory
wind_files = filter(x->occursin("weatherdata",x), readdir(wind_directory));

# remove the "." files
wind_files = wind_files[first.(wind_files,1) .!= "."]

#try

    # select the wind data filfie
    wind_data_file = pick_file(pwd() * "\\Wind_Data\\")

    # create a df to hold the wind data
    wind_data_df = CSV.read(wind_data_file, DataFrame);
#end

# get site name
wind_site_name = split(uppercase.(rsplit(wind_data_file,"\\"))[end],"_")[1]

# Read BoM weather station details (see http://www.bom.gov.au/qld/observations/map.shtml)
weather_station_df = CSV.read(".\\weather_stations.txt", DataFrame)

wind_latitude = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Latitude[1]
wind_longitude = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Longitude[1]
wind_height = weather_station_df[findall(==(wind_site_name), weather_station_df.Site),:].Height[1]

# Read wave buoy details (see https://www.qld.gov.au/environment/coasts-waterways/beach/monitoring/waves-sites)
wave_buoys_df = CSV.read(".\\wave_buoy_sites.txt", DataFrame)
wave_site_name
wave_latitude = wave_buoys_df[findall(==(wave_site_name), wave_buoys_df.Site),:].Latitude[1]
wave_longitude = wave_buoys_df[findall(==(wave_site_name), wave_buoys_df.Site),:].Longitude[1]
wave_depth = wave_buoys_df[findall(==(wave_site_name),wave_buoys_df.Site),:].Depth[1];

println("Done!")

### Calculate and plot bearing and distance between weather station and waverider buoy

In [ ]:
using Haversine

p1 = GeoLocation(λ=wind_longitude, ϕ=wind_latitude)
p2 = GeoLocation(λ=wave_longitude, ϕ=wave_latitude)

mid_lon = (p1.λ+p2.λ) / 2
mid_lat = (p1.ϕ+p2.ϕ) / 2

# returns heading in degrees
bearing = (HaversineBearing(p1, p2) + 360) % 360

# returns distance in meters
distance = HaversineDistance(p1, p2)

# get the bearing that is perpendicular to the bearing between to wind station and the buoy
dividing_bearing = bearing - 90
dividing_bearing = dividing_bearing < 0 ? dividing_bearing + 360 : dividing_bearing

#, label=wave_site_name 
@printf("The distance between the Wind Recorder and Waverider is %5.2fm\n",distance)
@printf("The bearing between the Wind Recorder and Waverider is %5.2fᵒ",bearing)

Plots.scatter([p1.λ, p2.λ],[p1.ϕ, p2.ϕ], mc=:blue, label="")
Plots.plot!([p1.λ, p2.λ],[p1.ϕ, p2.ϕ], lc=:blue, lw=:0.5, ls=:dash, label="")

# Add annotations for the point names
if p1.ϕ > p2.ϕ 
    Plots.annotate!([
        (p1.λ, p1.ϕ, Plots.text("  "*wind_site_name, :left, 8, :black, :left)),
        (p2.λ, p2.ϕ, Plots.text("  "*wave_site_name, :right, 8, :black, :right))])
else
    Plots.annotate!([
        (p1.λ, p1.ϕ, Plots.text("  "*wind_site_name, :right, 8, :black, :right)),
        (p2.λ, p2.ϕ, Plots.text("  "*wave_site_name, :left, 8, :black, :left))])
end  

brg_distance_text = "  Distance: "*string(round(distance/1000, digits=2))*"km\n  Bearing: "*string(round(bearing, digits=2))*"°"

Plots.annotate!([(mid_lon, mid_lat, Plots.text(brg_distance_text, :center, 8, :blue, :center))])

### Calculate and plot wind time lags between weather station and wave buoy

In [ ]:
#  calculate the effective wind speed component towards the Waverider buoy
function effective_wind_speed(wind_speed, wind_direction, bearing_to_point)
###########################################################################
    
    # Convert degrees to radians
    wind_dir_rad = deg2rad(wind_direction)
    bearing_rad = deg2rad(bearing_to_point)

    # Calculate the wind speed component in the direction of the bearing
    wind_speed_component = wind_speed * cos(wind_dir_rad - bearing_rad)
    
    return(wind_speed_component)
    
end    # effective_wind_speed()


# calculate the time lag between the anemometer and the Waverider
function calculate_time_lag(distance, wind_speed_component)
###########################################################

    if wind_speed_component <= 0
        return Inf  # Wind is not blowing towards the point, no meaningful time lag
    else
        time_lag = distance / wind_speed_component  # Time lag in hours
        return time_lag
    end

end    # calculate_time_lag()


# Process the wind data to calculate the time lag for each record
function calculate_time_lags(bearing, distance, wind_data_df)
#############################################################
   
    # Initialize an empty DataFrame to store the results
##    results = DataFrame(Date=wind_data_df.Date, Wind_Speed=wind_data_df.Wind_Speed, Wind_Direction=wind_data_df.Wind_Direction, Time_Lag=Float64[])
    results = DataFrame([[],[],[],[]], ["Date", "Wind_Speed", "Wind_Directio", "Time_Lag"])
    # Iterate over each row in the wind data DataFrame
    for row ∈ eachrow(wind_data_df)
        
        wind_speed = row.Wind_Speed / 3.6    # convert Km/Hr to m/s
        wind_direction = row.Wind_Direction
        
        # Calculate the effective wind speed component towards the second point
        wind_speed_component = effective_wind_speed(wind_speed, wind_direction, bearing)
        
        # Calculate the time lag
        time_lag = calculate_time_lag(distance, wind_speed_component)
        
        # Append the result to the results DataFrame
        push!(results, (row.Date, wind_speed, wind_direction, time_lag))
            
    end
    
    return(results)
        
end    # calculate_time_lags()


##################################################################################################
##################################################################################################
##################################################################################################
#==
Calls:
------
effective_wind_speed()
calculate_time_lag()
calculate_time_lags()

==#

# Calculate time lags
results_df = calculate_time_lags(bearing, distance, wind_data_df);

#println("Done!")

time_lag = []
U₁₀_vals = []
z_ref = wind_height    # height of anemometer
z_target = 10          # U₁₀
αₛₕₑₑᵣ = 0.143    # Wind sheer exponnent 0.1 over water

for ii ∈ eachrow(wind_data_df)

    wind_speed = ii.Wind_Speed
    
    # Convert wind speed recorded at anemometer to U₁₀ in m/s
    U₁₀ = wind_speed * (z_target / z_ref)^αₛₕₑₑᵣ
    push!(U₁₀_vals,U₁₀)
    
    wind_direction = ii.Wind_Direction
    
    if U₁₀ != 0
        
        lag = sind(wind_direction - dividing_bearing) * (distance/1000) / U₁₀
        
    else

        lag = NaN
        
    end
    
    push!(time_lag,lag)
    
end

# function to insert rows for complete dates
function fill_dates(df)
#######################
    
    min_date = df.Date[1]
    max_date = df.Date[end]

    complete_dates = min_date:Minute(30):max_date

    complete_df = DataFrame(Date = complete_dates)
    result_df = outerjoin(complete_df, df, on = :Date)

    sort!(result_df, :Date)
    
    return(result_df)
    
end    # fill_dates()


# Function to calculate the mean, excluding `nothing` and `missing` values
function safe_mean(arr)
#######################
    
    # Handle cases where `arr` might be `missing`
    if arr === missing
        return NaN
    end
    
    # Filter out `nothing` and `missing` values
    cleaned_arr = filter(x -> x !== nothing && x !== missing, arr)
    
    # Calculate the mean of the cleaned array
    if isempty(cleaned_arr)
        return NaN
    else
        return mean(cleaned_arr)
    end

end    # safe_mean()


result_df = fill_dates(wave_data_df)

# Apply the function to each row's Dirn column
mean_dir = [safe_mean(row.Dirn) for row in eachrow(result_df)]

#mean_dir = mean.([replace(x, nothing => NaN) for x ∈ result_df.Dirn])

# Get date limits
start_date = DateTime.(split(split(hxv_files[1],"_")[end],"T")[1]*" "*displacement_df.Time_string[1],"yyyy-mm-dd HH:MM")
end_date = DateTime.(split(split(hxv_files[1],"_")[end],"T")[1]*" "*displacement_df.Time_string[end],"yyyy-mm-dd HH:MM")

tm_tick = range(start_date, end_date, step=Hour(1))
ticks = Dates.format.(tm_tick,"HH:MM")

yrange = span([x for x ∈ wind_data_df.Wind_Speed[start_date .<= wind_data_df.Date .< end_date] if !isnan(x)])
p1 = Plots.plot(wind_data_df.Date, wind_data_df.Wind_Speed, yaxis="Wind speed (km/Hr)", xlims=(start_date,end_date), ylims=[yrange[1],yrange[end]], label="Anemometer Wind Speed at $wind_height m (km/Hr)", legend=:topleft)
p1 = Plots.plot!(wind_data_df.Date, U₁₀_vals, label="U₁₀ Wind Speed (km/Hr)")
subplot = Plots.twinx()
p1 = Plots.plot!(subplot, result_df.Date, result_df.Hm0, yaxis="Wave Height (m)", label="Hₘ₀ (m)", lc=:green, lw=:0.5, legend=:topright)

p2 = Plots.plot(wind_data_df.Date, wind_data_df.Wind_Direction, yaxis="Wind dir. (ᵒ)", label="Wind Direction (ᵒ)", yflip=:true, xlims=(start_date,end_date), ylims=(0,360), yticks=(0:45:360), legend=:bottomleft)
p2 = Plots.plot!(result_df.Date, mean_dir, label="Mean wave direction (ᵒ)", lc=:green, legend=:topright)

yrange = span([x for x ∈ time_lag[start_date .<= wind_data_df.Date .< end_date] if !isnan(x)])
p3 = Plots.plot(wind_data_df.Date, time_lag, yaxis="Lag (Hr)", xlims=(start_date,end_date), ylims=[yrange[1],yrange[end]], label="Arrival Time Lag (Hr)")
p3 = Plots.hline!([0],lc=:lightgrey, ls=:dash, label="")

p123_plot = Plots.plot(p1, p2, p3, size=(1500,800), layout=(3,1), 
    xlabel="Date", xlims=(start_date, end_date), xticks=(tm_tick,ticks), xtickfontsize=7, ylabelfontsize=:10,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent,
    leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, topmargin = 2Plots.mm, bottommargin = 3Plots.mm, title="")

try

    plot_file = ".\\Plots\\"*split(split(hxv_files[1],"\\")[end],"T")[1]*"_wind_time_lags_plot.png"
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)

catch

    "Alert: Plot not saved!"

end

display(p123_plot)

### Calculate wave time lags between weather station and wave buoy

In [ ]:
##################################################################################################
##################################################################################################
##################################################################################################

peak_dir = []
time_lag = []
wave_length = []
wave_speed = []

for ii in 1:nrow(displacement_df)
    
    # locate the peak frequency
    spec_peak = argmax(displacement_df.Chh[ii])
    
    # Calculate the period and direction of the spectral peak
    fₚ = displacement_df.fhh[ii][spec_peak]
    Tₚ = 1/fₚ
    ##println("Tₚ is $Tₚ seconds.")
    
    Pkdir = displacement_df.Direction[ii][spec_peak]
    ##println("Pkdir is $Pkdir ⁰")
    push!(peak_dir,Pkdir)
    
    using Roots
    
    T = Tₚ  # wave period in seconds
    d = wave_depth  # water depth in metres
    
    # Function to solve for wave number
    ff_wave_number(k) = g*k*tanh(k*d) - ((2π)/T)^2
    
    # Solve for wave number using the fzero function from the Roots package
    k = fzero(ff_wave_number, 0.01)  # initial guess for k is 0.01
    
    # Calculate wavelength
    λ = (2π)/k
    push!(wave_length, λ)
    
    # Calculate wave speed
    C = λ/T
    push!(wave_speed, C)
    
    # Calculate time lag in minutes between buoy and wind recorder
    lag = sind(Pkdir - dividing_bearing) * distance / C / 60

    push!(time_lag,lag)

##    @printf("%s Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Wave length = %6.2fm; Wave speed = %6.2fm/s; Lag = %6.2fminutes\n",
##            Dates.format(displacement_df.Date[i], "yyyy-mm-dd HH:MM"), Tₚ, Pkdir, λ, C, lag)
    

end

println("Done!")


### Plot time lags between Wind recorder and Wave buoy

In [ ]:
using Plots

# Sample data
x = dates
y = time_lag

# get wind data
wind_date = wind_data_df.Date
wind_speed = wind_data_df.Wind_Speed
wind_direction = wind_data_df.Wind_Direction

limit = maximum(abs.(time_lag))
# Plot the data

tm_tick = range(dates[1],dates[end],step=Day(2))
ticks = Dates.format.(tm_tick,"dd/mm")

p1 = Plots.plot(x, y, label="", fill=(0, :green), fillalpha=0.5, bottommargin = 0Plots.mm)
p1 = Plots.ylims!(0, limit)

p2 = Plots.plot(x, y, xlabel="x", label="", fill=(0, :red), fillalpha=0.5, topmargin = -8.4Plots.mm, bottommargin=7Plots.mm)
Plots.ylims!(-limit, 0)

p12 = Plots.plot(p1,p2,layout=(2,1), size=(1500,300), xlabel="Date", xlims=(dates[1],dates[end]), xticks=(tm_tick,ticks), xtickfontsize=7,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent,
    leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, title="")

display(p12)
#######################################################################################################
p1 = Plots.plot(dates,peak_dir, lc=:blue, ylabel="Peak Dir (ᵒ)", yflip=:true, label="Wave direction")
p1 = Plots.plot!(wind_date,wind_direction, lc=:red, label="Wind Direction")

p2 = Plots.plot(dates,wave_speed, ylabel="Speed (m/s)", label="Wave speed")
p2 = Plots.plot!(wind_date,wind_speed./3.6, lc=:red, label="Wind speed")

p3 = Plots.plot(dates,wave_length, ylabel="Wave length (metres)", label="")
p3 = Plots.hline!([wave_depth*2], label="Wave Base")


p123_plot = Plots.plot(p1, p2, p3, size=(1500,600), layout=(3,1), 
    xlabel="Date", xlims=(dates[1],dates[end]), xticks=(tm_tick,ticks), xtickfontsize=7,
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent,
    leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, topmargin = 2Plots.mm, bottommargin = 5Plots.mm, title="")

display(p123_plot)

### Merge the two df's into a single df on Date

In [ ]:
using DataFrames, Plots, Dates

merged_df = innerjoin(wind_data_df, wave_data_df, on = :Date);

println("Done!")

### Use nearest-neighbour routine to calculate best track of directions

In [ ]:
using LinearAlgebra    

# use a nearest neighbour approach to locate the closest point
function nearest_neighbor(target, points)
#########################################

    distances = [norm(target - point) for point in points]
    nearest_index = argmin(distances)
    
    return points[nearest_index]
    
end    # nearest_neighbor()


function correct_directions(direction)    
######################################
    
    new_dir = []

    current = direction[1]
    nearest = current

    for next ∈ direction[2:end]

        push!(new_dir,current)
        
        if next == nothing # this will probably fail if first direction is nothing
            
            next = nearest
            
        end
        
        # create three points to cover full range from 0-360⁰
        points = [next - 360, next, next + 360]

        # now find the point nearest to current point
        nearest = nearest_neighbor(current, points)
        current = nearest

    end

    push!(new_dir,current)
    new_dir = Float32.(Float32.(coalesce.(new_dir, NaN)))
    
    return(new_dir)
    
end    # correct_directions()
    

##################################################################################################
##################################################################################################
##################################################################################################

corrected_wind_direction = correct_directions(merged_df.Wind_Direction)

using StatsBase

weighted_mean_direction = []
peak_dirn = []

# Define a function that checks for NaN or Nothing
isnothingornan(x) = x === nothing || (isa(x, Number) && isnan(x))

for ii ∈ 1:nrow(merged_df)
    
    peak = argmax(merged_df.Chh[ii])
    push!(peak_dirn, merged_df.Dirn[ii][peak])
    
    #find index of all frequencies in range from 0.05 to 1.0 Hz
    global dir_range = findall(x -> 0.05 <= x <= 1.0, merged_df.fhh[ii])

    # get mean direction (weighted by spectral energy values)    
    dir_vals = merged_df.Dirn[ii][dir_range]
    weight_vals = Float64.(merged_df.Chh[ii][dir_range])
    
    # need to test for NaN's or Nothing in the data and remove them before tryinh to find weighted mean
    index = findall(isnothingornan, dir_vals)
    if !isempty(index) 
        dir_vals = deleteat!(dir_vals, index)
        weight_vals = deleteat!(weight_vals[:,1], index)
    end
    
#    println(ii,"  ",mean(merged_df.Dirn[ii][dir_range],"  ",Weights(Float64.(merged_df.Chh[ii][dir_range]))))
    push!(weighted_mean_direction, mean(dir_vals, Weights(weight_vals)))

end
    
corrected_mean_direction = correct_directions(weighted_mean_direction);
corrected_peak_direction = correct_directions(peak_dirn);

println("Done!")

### Plot Wind speed v's Hm0

In [ ]:
# remove possible spikes
merged_df.Hm0[findall(merged_df.Hm0 .> 5)] .= NaN;
merged_df.Tp[findall(merged_df.Tp .> 20)] .= NaN;

println("Done!")

### Calculate Hm0 and Tp for sea and swell

In [ ]:
using LinearAlgebra

using Peaks
using Plots
using StatsBase
using Printf
using Tk
using LsqFit

function get_wave_types1(t, y, previous_wave_type, separation_point)
####################################################################
    
"""
    Calls:  calc_moments()
            nearest_neighbor()
    Called by: Main
"""
    αₚₘ = 0.0081
    peak = argmax(y)
    fₚ = t[peak]
    Tₚ = 1/fₚ    
    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t,y)
    hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc = calc_parameters(m₋₁, m₀, m₁, m₂, m₄)
    
    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t]

    # determine whether this part of wave record is sea or swell
    # Refer https://www.researchgate.net/publication/249605099_Spectral_Partitioning_and_Identification_of_Wind_Sea_and_Swell
    ratio = y[peak] / Sf[peak]
    wave_type = (ratio < 1 ? "Ocean Swell" : "Wind Sea")

    # test whether separation between sea and swell has been found
    if (previous_wave_type == "Ocean Swell") && (wave_type == "Wind Sea")
    # Separation between swell and sea has been located
        
        separation_point = t[1]
        
    end

    return(wave_type, separation_point)
        
end    # get_wave_types1()


function calc_parameters(m₋₁, m₀, m₁, m₂, m₄)
#############################################
    
    hₘ₀ = 4√m₀
    Hᵣₘₛ = √(8m₀)
    T₀₂ = √(m₀/m₂)    # mean wave period
    T₀₁ = m₀/m₁       # significant wave period
    T₋₁₀ = m₋₁/m₀     # mean energy period Tₑ
    Tc = √(m₂/m₄)

    return(hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc)

    end    # calc_parameters()


function get_hm0_tp_pkdir_meandir(t_, y_, dir, time_string)
###########################################################
    
    αₚₘ = 0.0081    
    peak = argmax(y_)
    fₚ = t_[peak]
    Tₚ = 1/fₚ 

    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t_,y_)
    hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc = calc_parameters(m₋₁, m₀, m₁, m₂, m₄)

    # get peak direction and mean direction (weighted by spectral energy values)
    peak_direction = dir[peak]
    weighted_mean_direction = mean(dir, Weights(y_))

    # Calculate representative P-M spectra
    Sf = [αₚₘ*g^2 * (2π)^-4 * ff^-5 * exp(-1.25 * (ff/fₚ)^-4) for ff ∈ t_]
    Sf = replace!(Sf, Inf=>NaN)
    
    ratio = y_[peak] / Sf[peak]    # refer Portilla et al (2009) p.117 Section 2.
    wave_type = (ratio < 1 ? "Ocean Swell" : "Wind Sea")

##    @printf("%s Hₘ₀ = %5.2fm; Hᵣₘₛ = %5.2fm; T₋₁₀ = %5.2fs; T₀₁ = %5.2fs; T₀₂ = %5.2fs; Tc = %5.2fs; Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Mean Dirn = %6.2fᵒ Wave type = %s; Ratio = %2.4f\n",
##        time_string, hₘ₀, Hᵣₘₛ, T₋₁₀, T₀₁, T₀₂, Tc, Tₚ, peak_direction, weighted_mean_direction, wave_type, ratio)
###    @printf("%s Hₘ₀ = %5.2fm; Tₚ = %5.2fs; Peak Dirn = %6.2fᵒ; Mean Dirn = %6.2fᵒ Wave type = %s; Ratio = %2.4f\n",
###        time_string, hₘ₀, Tₚ, peak_direction, weighted_mean_direction, wave_type, ratio)
    flush(stdout)

    return(hₘ₀, Tₚ, peak_direction, weighted_mean_direction, Sf, wave_type)

    end    # get_hm0_tp_pkdir_meandir()


# function to eliminate minor peaks where the trough between adjacent peaks is greater than 50% of the smaller peak    
function eliminate_minor_peaks(x_peaks, y_peaks, x_troughs, y_troughs)
######################################################################
 
"""
    Calls: Nil
    Called by: find_peaks_and_valleys()
"""
    
    # Initialize new arrays to store significant peaks and troughs
    x_peaks_significant = Float64[]
    y_peaks_significant = Float64[]
    x_troughs_significant = Float64[]
    y_troughs_significant = Float64[]

    # Loop over troughs (the number of veal troughs is 1 less than the number of real peaks)
    for i in 1:length(x_troughs)

        # Check if trough is less than 50% of the smaller adjacent peaks
        if y_troughs[i]*2 < min(y_peaks[i], y_peaks[i+1])

            push!(x_peaks_significant, x_peaks[i], x_peaks[i+1])
            push!(y_peaks_significant, y_peaks[i], y_peaks[i+1])
            push!(x_troughs_significant, x_troughs[i])
            push!(y_troughs_significant, y_troughs[i]) 

        end
        
    end

    return(x_peaks_significant, y_peaks_significant, x_troughs_significant, y_troughs_significant)

end    # eliminate_minor_peaks()


# remove nearby lesser peaks
function filter_spectral_values(peak_values, peak_freqs)
########################################################
    
    filtered_peak_values = Float64[]  # Initialize an empty array for filtered values
    filtered_peak_freqs = Float64[]   # Initialize an empty array for filtered frequencies
    n = length(peak_values)

    for i in 1:n-1
        freq_diff = peak_freqs[i+1] - peak_freqs[i]
        if freq_diff > 0.02
            push!(filtered_peak_values, peak_values[i])
            push!(filtered_peak_freqs, peak_freqs[i])
        end
    end

    # Add the last value (since there's no next value to compare)
    push!(filtered_peak_values, peak_values[n])
    push!(filtered_peak_freqs, peak_freqs[n])

    return(filtered_peak_values, filtered_peak_freqs)

end    # filter_spectral_values()


function find_peaks_and_valleys(t, y, peak_vals)
################################################
    
"""
    Calls: eliminate_minor_peaks()
    Called by: Main
"""    

    freqs = t
    spectrum = y

    maxima = findmaxima(spectrum)
    pidx = maxima[1]
    peak_values = maxima[2]
    peak_freqs = freqs[pidx]
    
#    peak_freqs,peak_values =  filter_spectral_values(peak_values, peak_freqs)

    # Set a minimum value (20% of fp value) and only accept peaks above this
    max_peak = maximum(peak_vals)
    cutoff = max_peak * 0.02
##    println(cutoff)

    # Only keep peaks above cutoff
    filtered_peak_indices = findall(x -> x >= cutoff, peak_values)
    filtered_peak_values = peak_values[filtered_peak_indices]
    filtered_peak_freqs = peak_freqs[filtered_peak_indices]

    # Merge peaks
    merged_freqs = []
    merged_values = []
    tolerance = 0.05
    for i in 1:length(filtered_peak_freqs)
        if isempty(merged_freqs) || minimum(abs.(filtered_peak_freqs[i] .- merged_freqs)) > tolerance
            push!(merged_freqs, filtered_peak_freqs[i])
            push!(merged_values, filtered_peak_values[i])
        else
            for j in 1:length(merged_freqs)
                if abs(filtered_peak_freqs[i] - merged_freqs[j]) <= tolerance
                    if filtered_peak_values[i] > merged_values[j]
                        merged_freqs[j] = filtered_peak_freqs[i]
                        merged_values[j] = filtered_peak_values[i]
                    end
                end
            end
        end
    end

    # Finding valleys/separation points between merged peaks
    separation_freqs = []
    separation_values = []
    for i in 2:length(merged_freqs)
        # Check interval between two successive peaks
        interval_idx = findall(x -> x>=merged_freqs[i-1] && x<=merged_freqs[i], freqs) 

        if !isempty(interval_idx)
            interval_spectrum = spectrum[interval_idx]
            minima = findminima(interval_spectrum)

            # find minimum value in valleys in the separation point
            if !isempty(minima[1])
                # findminima returns all minima in interval. We only want the lowest.
                min_val_index = argmin(minima[2])
                valley_idx = interval_idx[minima[1][min_val_index]]
                push!(separation_freqs, freqs[valley_idx])
                push!(separation_values, minima[2][min_val_index])
            end
        end
    end
    
    merged_freqs, merged_values, separation_freqs, separation_values = eliminate_minor_peaks(merged_freqs, merged_values, separation_freqs, separation_values)

    return(merged_freqs, merged_values, separation_freqs, separation_values, cutoff)
    
end    # find_peaks_and_valleys()


# function to calc spectral moments
function calc_moments(t,y)
##########################
    
"""
    Calls: Nil
    Called by: calc_PM_JONSWAP()
"""
    
    ax1 = (last(t) - first(t)) / (length(t)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s_01 = 0; s00 = 0; s01 = 0; s02 = 0; s03 = 0; s04 = 0;
    m₋₁ = 0; m₀ = 0; m₁ = 0; m₂ = 0; m₃ = 0; m₄ = 0

    for ii in 1:length(t)

        s_01 += t[ii]^-1 * y[ii]
        s00 += t[ii]^0 * y[ii]
        s01 += t[ii]^1 * y[ii]
        s02 += t[ii]^2 * y[ii]
        s03 += t[ii]^3 * y[ii]
        s04 += t[ii]^4 * y[ii]

    end

    m₋₁ = 0.5*ax1*(first(t)^-1*first(y) + 2*s_01 + last(t)^0*last(y))
    m₀ = 0.5*ax1*(first(t)^0*first(y) + 2*s00 + last(t)^0*last(y))
    m₁ = 0.5*ax1*(first(t)^1*first(y) + 2*s01 + last(t)^1*last(y))
    m₂ = 0.5*ax1*(first(t)^2*first(y) + 2*s02 + last(t)^2*last(y))
    m₃ = 0.5*ax1*(first(t)^3*first(y) + 2*s03 + last(t)^3*last(y))
    m₄ = 0.5*ax1*(first(t)^4*first(y) + 2*s04 + last(t)^4*last(y))        

    return(m₋₁, m₀, m₁, m₂, m₄)

    end    # calc_moments()


#################################################################################################
#################################################################################################
#################################################################################################

# define a df to hold wind sea spectral values
wind_sea_df = DataFrame([[], [], [], [], []], ["Date", "Frequency", "Spectra", "PM", "Direction"])


# define arrays to hold sea and swell data
global hm0_swell_array = []; Tp_swell_array = []; pkdir_swell_array = []; mean_dir_swell_array = []
global hm0_sea_array = []; Tp_sea_array = []; pkdir_sea_array = []; mean_dir_sea_array = []
hm0_all_array = []; Tp_all_array = []; pkdir_all_array = []; mean_dir_all_array = []

for iii in 1:nrow(displacement_df)
    
    t = displacement_df.fhh[iii]
    y = displacement_df.Chh[iii]

    dir = displacement_df.Direction[iii]
    spread = displacement_df.Spread[iii]

    # only use data in the frequency range 0.03 - 0.6 Hz.
    valid = findall(0.03 .< t .< 0.6)
    t = t[valid]
    y = y[valid]
    dir = dir[valid]
    spread = spread[valid] 
      
    # Initialize starting variables
    start = 1
    separation_point = 0
    wave_type = "?"
    previous_wave_type = "?"
    

    # locate ALL peaks and the separation points between them
    peaks, peak_vals = findmaxima(y);
    merged_freqs, merged_values, separation_freqs, separation_values, cutoff = find_peaks_and_valleys(t, y, peak_vals)

    # get the wave parameters
    hm0, Tp, pkdir, mean_dir, Sf, wave_type_all = get_hm0_tp_pkdir_meandir(t, y, dir, dates[iii])
    push!(hm0_all_array, hm0); push!(Tp_all_array, Tp); push!(pkdir_all_array, pkdir); push!(mean_dir_all_array, mean_dir)

    # If there are no separation points, it means we have a Unimodal wave event
    if isempty(separation_freqs)   
        
        # populate the sea and swell arrays depending on where unimodal event is either sea or swell - NaN's will go to the other arrays
        if wave_type_all == "Ocean Swell"

            push!(hm0_swell_array, hm0); push!(Tp_swell_array, Tp); push!(pkdir_swell_array, pkdir); push!(mean_dir_swell_array, mean_dir)
            push!(hm0_sea_array, NaN); push!(Tp_sea_array, NaN); push!(pkdir_sea_array, NaN); push!(mean_dir_sea_array, NaN)
            
        elseif wave_type_all == "Wind Sea"
                       
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [dates[iii], t, y, Sf, dir])
            
        else
            
            # should not happen!
            println("Unusual Unimodal event detected")
            
        end
            
        
    else
        
        # multimodal event, so repeat for each separation frequency detected
        for ii in separation_freqs
            
            next = findfirst(x->x==ii, t)
            wave_type, separation_point = get_wave_types1(t[start:next], y[start:next], previous_wave_type, separation_point)

            # Identify if a sea-swell separation has occurred
            if ((previous_wave_type=="Ocean Swell") && (wave_type=="Wind Sea"))
                
                break    # Exit the for loop

            end

            start = next         
            previous_wave_type = wave_type
            
        end
        
        hm0_swell, Tp_swell, pkdir_swell, mean_dir_swell, Sf_swell, wave_type_swell = get_hm0_tp_pkdir_meandir(t[1:start], y[1:start], dir[1:start], dates[iii])
        hm0_sea, Tp_sea, pkdir_sea, mean_dir_sea, Sf_sea, wave_type_sea = get_hm0_tp_pkdir_meandir(t[start:end], y[start:end], dir[start:end], dates[iii])
        

        if ((wave_type_swell=="Ocean Swell") & (wave_type_sea=="Wind Sea"))
            
            push!(hm0_swell_array, hm0_swell); push!(Tp_swell_array, Tp_swell); push!(pkdir_swell_array, pkdir_swell); push!(mean_dir_swell_array, mean_dir_swell)
            push!(hm0_sea_array, hm0_sea); push!(Tp_sea_array, Tp_sea); push!(pkdir_sea_array, pkdir_sea); push!(mean_dir_sea_array, mean_dir_sea)
            
            push!(wind_sea_df, [dates[iii], t[start:end], y[start:end], Sf_sea, dir[start:end]])
            
        elseif ((wave_type_swell=="Ocean Swell") & (wave_type_sea=="Ocean Swell"))
            
            push!(hm0_swell_array, hm0); push!(Tp_swell_array, Tp); push!(pkdir_swell_array, pkdir); push!(mean_dir_swell_array, mean_dir)
            push!(hm0_sea_array, NaN); push!(Tp_sea_array, NaN); push!(pkdir_sea_array, NaN); push!(mean_dir_sea_array, NaN)
            
        elseif ((wave_type_swell=="Wind Sea") & (wave_type_sea=="Wind Sea"))
            
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [dates[iii], t[start:end], y[start:end], Sf_sea, dir[start:end]])
            
        else # must be a Wind and Swell combination - this could be an artifact of the PM spectrum
            
            push!(hm0_swell_array, NaN); push!(Tp_swell_array, NaN); push!(pkdir_swell_array, NaN); push!(mean_dir_swell_array, NaN)
            push!(hm0_sea_array, hm0); push!(Tp_sea_array, Tp); push!(pkdir_sea_array, pkdir); push!(mean_dir_sea_array, mean_dir)
            
            push!(wind_sea_df, [dates[iii], t[start:end], y[start:end], Sf_sea, dir[start:end]])
            
        end
    
    end

#    println(dates[iii]*" "*wave_type_swell*" "*wave_type_sea)
    
##    println("----------------------------------------------------------------------------------------------------------")

end

# create a new df to hold the sea and swell values
sea_swell_df = DataFrame(Date = dates)
sea_swell_df.Sea = hm0_sea_array
sea_swell_df.Pkdir_Sea =  pkdir_sea_array
sea_swell_df.mean_dir_Sea =  mean_dir_sea_array

sea_swell_df.Swell = hm0_swell_array
sea_swell_df.Pkdir_Swell =  pkdir_swell_array
sea_swell_df.Mean_Dir_Swell =  mean_dir_swell_array;

println("Done!")

### Show Hm0 and Tp for sea and swell

In [ ]:
# Plot Hm0 and Tp for sea and swell
dates = DateTime.([(split.(el, "_")[2])[1:end-4] for el in hxv_files], "yyyy-mm-ddTHHhMMK")

# build title string showing date range
title=split(split(hxv_files[1],"\\")[end],"T")[1]
##annotation_string = split(hxv_directory, "\\")[3]*"_"*title

p1 = Plots.plot(dates, hm0_all_array, lc=:blue, lw=:3, label="Total", ylabel="Hm0 (m)", title=title)
p1 = Plots.plot!(sea_swell_df.Date, sea_swell_df.Sea, lc=:red, lw=:3, label="Sea")
p1 = Plots.plot!(sea_swell_df.Date, sea_swell_df.Swell, lc=:green, lw=:3, label="Swell")

#p2 = Plots.plot(dates,Tp_all_array, lc=:blue, lw=:2, label="Total", ylabel="Tp (s)")
p2 = Plots.plot(dates, Tp_sea_array, lc=:red, lw=:3, label="Sea", ylabel="Tp (s)")
p2 = Plots.plot!(dates, Tp_swell_array, lc=:green, lw=:3, label="Swell")

p3 = Plots.plot(dates, mean_dir_all_array, lc=:blue, lw=:3, label="Total", yflip=:true, ylabel="Mean direction (ᵒ)", legend=:bottomright)
p3 = Plots.plot!(dates, mean_dir_sea_array, lc=:red, lw=:3, label="Sea")
p3 = Plots.plot!(dates, mean_dir_swell_array, lc=:green, lw=:3, label="Swell")

p4 = Plots.plot(dates, pkdir_all_array, lc=:blue, lw=:3, label="Total", yflip=:true, ylabel="Peak direction (ᵒ)", xlabel="Time AEST", legend=:bottomright)
p4 = Plots.plot!(dates, pkdir_sea_array, lc=:red, lw=:3, label="Sea")
p4 = Plots.plot!(dates, pkdir_swell_array, lc=:green, lw=:3, label="Swell")

#p1 = Plots.annotate!(dates[1], maximum(hm0_all_array), (annotation_string, :left, :blue, 10))

tm_tick = range(dates[1],dates[end],step=Hour(1))
ticks = Dates.format.(tm_tick,"HH")

p1_plot = Plots.plot(p1, p2, p3, p4, xlims=(dates[1],dates[end]), xticks=(tm_tick,ticks), xtickfontsize=7,
    ytickfontsize=8, 
    titlefontsize=10, framestyle = :box, guidefontsize=10, 
    fg_legend=:transparent, bg_legend=:transparent, plot_padding=1Plots.mm,
    leftmargin = 15Plots.mm, bottommargin = 5Plots.mm, grid=true, size=(1600,800), layout=(4,1), gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

try
#    plot_date = Dates.format(displacement_df.Date[iii], "yyyy_mm")
    plot_file = ".\\Plots\\"*split(split(hxv_files[1],"\\")[end],"T")[1]*"_daily_parameter_plot.png"
    savefig(plot_file)
    println("\nPlot file saved as "*plot_file)
catch
    "Alert: Plot not saved!"
end

Plots.display(p1_plot)

### Need to correct the wind sea directions (weighted by wind sea spectra)

In [ ]:
using StatsBase

weighted_mean_direction_sea = []
peak_dirn_sea = []

# Define a function that checks for NaN or Nothing
isnothingornan(x) = x === nothing || (isa(x, Number) && isnan(x))

for ii ∈ 1:nrow(wind_sea_df)
    
    peak = argmax(wind_sea_df.Spectra[ii])
    push!(peak_dirn_sea, wind_sea_df.Direction[ii][peak])
    
    #find index of all frequencies in range from 0.05 to 1.0 Hz
    global dir_range = findall(x -> 0.05 <= x <= 1.0, wind_sea_df.Frequency[ii])

    # get mean direction (weighted by spectral energy values)    
    dir_vals = wind_sea_df.Direction[ii][dir_range]
    weight_vals = Float64.(wind_sea_df.Spectra[ii][dir_range])
    
    # need to test for NaN's or Nothing in the data and remove them before tryinh to find weighted mean
    index = findall(isnothingornan, dir_vals)
    if !isempty(index) 
        dir_vals = deleteat!(dir_vals, index)
        weight_vals = deleteat!(weight_vals[:,1], index)
    end
    
##    println(ii,"  ",mean(wind_sea_df.Direction[ii][dir_range],"  ",Weights(Float64.(wind_sea_df.Spectra[ii][dir_range]))))
    push!(weighted_mean_direction_sea, mean(dir_vals, Weights(weight_vals)))

end
    
corrected_mean_direction_sea = correct_directions(weighted_mean_direction_sea);
corrected_peak_direction_sea = correct_directions(peak_dirn_sea);

println("Done!")

### Plot wave heights, wind speeds, wind and wave directions

In [ ]:
using EasyFit

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

merged_df = coalesce.(merged_df, NaN)

# Define height or wind speed recorder
xx = wind_height # 100m for Cape Moreton
Uxx = merged_df.Wind_Speed

# Convert Uxx to U₁₀ in m/s
merged_df.U₁₀ = Uxx .* log(10/z₀) ./ log(xx/z₀)  ./ 3.6    # convert to m/s

ma = movavg(merged_df.U₁₀, 5)
U₁₀_observed = ma.x
p1 = Plots.plot(merged_df.Date, U₁₀_observed, lw=:0.5, lc=:blue, grid=:true, label="U₁₀ Wind speed (smoothed)", ylabel="Wind speed (m/s)", ylims=(0,15), legend=:topleft, title="Wind speed v's Hₘ₀")

subplot = Plots.twinx()
p1 = Plots.plot!(subplot, merged_df.Date, merged_df.Hm0, lw=:0.5, lc=:red, label="Hₘ₀", ylabel="Hₘ₀ (m)", ylim=(0,maximum(merged_df.Hm0)*1.05), legend=:topright)

Hₘ₀_U₁₀ = 0.243 .* (merged_df.U₁₀).^2 ./ g
m_Hₘ₀ = movavg(Hₘ₀_U₁₀, 1)

p1 = Plots.plot!(subplot, merged_df.Date, m_Hₘ₀.x, lw=:0.5, lc=:lightgrey, label="Hₘ₀ from U₁₀", ylabel="Hₘ₀ (m)", ylim=(0,maximum(merged_df.Hm0)*1.05), legend=:topright)
p1 = Plots.plot!(sea_swell_df.Date, sea_swell_df.Sea, lc=:green, label="Wind Sea")

### Plot Wind speed v's Tp
p2 = Plots.plot(merged_df.Date, ma.x, w=:0.5, lc=:blue, grid=:true, label="U₁₀ Wind speed (smoothed)", ylabel="U₁₀ Wind speed (m/s)", ylims=(0,15), legend=:topleft, title="Wind speed v's Tₚ")

subplot = Plots.twinx()
mb = movavg(merged_df.Tp, 1)
p2 = Plots.plot!(subplot, merged_df.Date, mb.x, w=:0.5, lc=:red, label="Tₚ (smoothed)", ylabel="Tₚ (s)", ylim=(0,15), legend=:topright)

### Plot wind directions
ma = movavg(corrected_wind_direction, 1)
p3 = Plots.plot(merged_df.Date, corrected_wind_direction, w=:0.5, lc=:blue, label="Wind direction", ylims=(0,360), yflip=:true, legend=:bottomright, title="Wind and Wave directions", ylabel="Direction (⁰)")
p3 = Plots.plot!(merged_df.Date, corrected_wind_direction.+360, w=:0.5, lc=:blue, label="")
p3 = Plots.plot!(merged_df.Date, corrected_wind_direction.-360, w=:0.5, lc=:blue, label="")

p3 = Plots.plot!(wind_sea_df.Date, corrected_mean_direction_sea, w=:0.5, lc=:red, label="Mean Wave direction")
p3 = Plots.plot!(wind_sea_df.Date, corrected_mean_direction_sea.+360, w=:0.5, lc=:red, label="")
p3 = Plots.plot!(wind_sea_df.Date, corrected_mean_direction_sea.-360, w=:0.5, lc=:red, label="")

p3 = Plots.plot!(wind_sea_df.Date, corrected_peak_direction_sea, w=:0.5, lc=:pink, label="Peak Wave direction")
p3 = Plots.plot!(wind_sea_df.Date, corrected_peak_direction_sea.+360, w=:0.5, lc=:pink, label="")
p3 = Plots.plot!(wind_sea_df.Date, corrected_peak_direction_sea.-360, w=:0.5, lc=:pink, label="")

tm_tick = range(merged_df.Date[1],merged_df.Date[end],step=Hour(2))
ticks = Dates.format.(tm_tick,"HH:MM");

plot_p1 = Plots.plot(p1, p2, p3, size=(1600,800), layout=(3,1), framestyle = :box, fg_legend=:transparent, bg_legend=:transparent, xlims=(merged_df.Date[1],merged_df.Date[end]), legendfontpointsize=:10,
    suptitle = wave_site_name*" waverider and "*wind_site_name*" anemometer "*Dates.format(merged_df.Date[1], "yyyy-mm-dd HH:MM")*" to "*Dates.format(merged_df.Date[end], "yyyy-mm-dd HH:MM"),xticks=(tm_tick,ticks),
    leftmargin = 15Plots.mm, rightmargin = 15Plots.mm, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1)

try
    # Output plot file name
    plt_file = ".\\Plots\\"*split(split(hxv_files[1],"\\")[end],"T")[1]*"_wind_and_wave_plot.png"
    savefig(plot_p1, plt_file)
    println("\nPlot file saved as ",plt_file)
catch
    "Alert: Plot not saved!"
end

display(plot_p1)

In [ ]:
function estimate_Uz(frequency, spectra)
################################################
    
    # Constants
    αₖ = 0.012  # Charnock constant
    z = 10
    
    # Calculate friction velocity (uₛₜₐᵣ) from wave spectra
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1)
    
    # Estimate roughness length (z₀) using Charnock relation
##    z₀ = αₖ * uₛₜₐᵣ^2 / g
    
    # Calculate vertical profile of horizontal wind velocity (Uz)
    Uz = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]

    @printf("Uₛₜₐᵣ estimate = %5.4f m/s\n",uₛₜₐᵣ)
    @printf("z₀ estimate = %5.4f m\n",z₀)
    
    return(Uz)
                    
end    # convert_to_probability_distribution()

##################################################################################################
##################################################################################################
##################################################################################################

frequency = merged_df.fhh
spectra = merged_df.Chh
spread = merged_df.Spread

Uz = []

for iii in 1:nrow(merged_df)
                        
    Uz_estimate = mean(estimate_Uz(frequency[iii], spectra[iii], spread[iii]))
    push!(Uz,Uz_estimate)
##    println(merged_df.Date[iii], "  ", Uz_estimate)
                        
end

println("Done!")

In [ ]:
p1 = Plots.plot(merged_df.Date, merged_df.Hm0, label="Hm0")
p2 = Plots.plot(merged_df.Date, merged_df.U₁₀ .* 3.6, label="U₁₀ calc")
#p2 = Plots.plot!(merged_df.Date, U₁₀_observed, label="U₁₀ Observed")
p3 = Plots.plot(merged_df.Date, Uz, label="Uz calc")
     
plot_p123 = Plots.plot(p1, p2, p3, layout=(3,1), size=(1500,700), framestyle=:box, fg_legend=:transparent, bg_legend=:transparent, legendfontpointsize=:10,
    leftmargin=15Plots.mm, rightmargin=15Plots.mm, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, 
    title=title, titlefontsize=:12)

display(plot_p123)

### Calculate Uz values from the spectra

In [ ]:
using Statistics
using Plots

# function to calc spectral moments
function calc_moments(t,y)
##########################
    
"""
    Calls: Nil
    Called by: calc_PM_JONSWAP()
"""
    ax1 = (last(t) - first(t)) / (length(t)-1)

    # calc spectral moments m0, m1, m2, m3, and m4
    s_₀₁ = 0; s₀₀ = 0; s₀₁ = 0; s₀₂ = 0; s₀₃ = 0; s₀₄ = 0;
    m₋₁ = 0; m₀ = 0; m₁ = 0; m₂ = 0; m₃ = 0; m₄ = 0

    for ii in 1:length(t)

        s_₀₁ += t[ii]^-1 * y[ii]
        s₀₀ += t[ii]^0 * y[ii]
        s₀₁ += t[ii]^1 * y[ii]
        s₀₂ += t[ii]^2 * y[ii]
        s₀₃ += t[ii]^3 * y[ii]
        s₀₄ += t[ii]^4 * y[ii]

    end

    m₋₁ = 0.5*ax1*(first(t)^-1*first(y) + 2*s_₀₁ + last(t)^0*last(y))
    m₀ = 0.5*ax1*(first(t)^0*first(y) + 2*s₀₀ + last(t)^0*last(y))
    m₁ = 0.5*ax1*(first(t)^1*first(y) + 2*s₀₁ + last(t)^1*last(y))
    m₂ = 0.5*ax1*(first(t)^2*first(y) + 2*s₀₂ + last(t)^2*last(y))
    m₃ = 0.5*ax1*(first(t)^3*first(y) + 2*s₀₃ + last(t)^3*last(y))
    m₄ = 0.5*ax1*(first(t)^4*first(y) + 2*s₀₄ + last(t)^4*last(y))        

    return(m₋₁, m₀, m₁, m₂, m₄)

    end    # calc_moments()


# calculate frequency domain parameters            
function calc_parameters(t_,y_)
###############################

"""
    Calls: calc_moments
"""
            
    m₋₁, m₀, m₁, m₂, m₄ = calc_moments(t_,y_)
    hₘ₀ = 4√m₀
    Hᵣₘₛ = √(8m₀)
    T₀₂ = √(m₀/m₂)    # mean wave period
    T₀₁ = m₀/m₁       # significant wave period
    T₋₁₀ = m₋₁/m₀     # mean energy period Tₑ
    Tc = √(m₂/m₄)
            
    return(hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc)

    end    # calc_parameters()


# Calculate Uz at a given frequency using Phillips equilibrium formulation
function calculate_Uz_at_frequency(uₛₜₐᵣ, f, hₘ₀) 
###########################################
   
    αₛ = 0.03    # empirical constant - varies between  0.02 to 0.04
##    z₀ = αₛ*hₘ₀/2    # Charnock (1955)
      
    Uz = uₛₜₐᵣ / κ * log(10 / z₀)  # Calculate Uz at a reference height of 10 meters
    
    # Adjust Uz based on frequency, if necessary
    # You may need to incorporate additional factors or empirical relationships here
    
    return(Uz)

end    # calculate_Uz_at_frequency()


# Calculate integrated wave energy (I_p) from spectra
function calculate_integrated_wave_energy(spectra, frequencies)
###############################################################
    
    Ip = sum(spectra)
    
    return(Ip)

end    # calculate_integrated_wave_energy()


function estimate_wind_speed(spectra, frequency, hₘ₀)
##################################################

    # Calculate integrated wave energy (I_p) from spectra
##    Ip = calculate_integrated_wave_energy(spectra, frequency)

    # Calculate friction velocity (uₛₜₐᵣ) using the Charnock relation
    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)    # from Kaley Mudd et al (2024) p.3 Equation (4), and Thomson et al (2013) p.1 Equation (1) #calculate_friction_velocity(frequency, spectra, spread) # √(β * Ip * g)
#    @printf("Uₛₜₐᵣ estimate = %5.2f m/s\n",uₛₜₐᵣ)

    # Estimate vertical profile of horizontal wind velocity (Uz)
    Uz = [calculate_Uz_at_frequency(uₛₜₐᵣ, f, hₘ₀) for f in frequency]
    
    return(Uz)    
    
end    # estimate_wind_speed()


###################################################################################################
###################################################################################################
###################################################################################################

dates = DateTime.([(split.(el, "_")[2])[1:end-4] for el in hxv_files], "yyyy-mm-ddTHHhMMK")

# build title string showing date range
title = split(split(hxv_files[1],"\\")[end],"T")[1]*" "*Dates.format(dates[1], "U yyyy")

Uz = []
Hm0 = []

for i ∈ 1:nrow(wind_sea_df)
    
    frequencies = wind_sea_df.Frequency[i] 
    spectra = wind_sea_df.Spectra[i]  

    hₘ₀, Hᵣₘₛ, T₀₂, T₀₁, T₋₁₀, Tc = calc_parameters(frequencies, spectra)
    push!(Hm0, hₘ₀)

    Uz_estimate = estimate_wind_speed(spectra, frequencies, hₘ₀)
    push!(Uz, mean(Uz_estimate) / 3.6)
##    println(Dates.format(displacement_df.Date[i], "yyyy-mm-dd HH:MM:SS"), "  ", mean(Uz_estimate))

end

replace.(wind_sea_df[:, Between(:Spectra,:Direction)], Inf=>NaN);

# Convert Uxx to U₁₀
xx = wind_height
Uxx = wind_data_df.Wind_Speed

# Define the roughness length
##z₀ = 0.002 # for open water
αₛ = 0.03    # empirical constant - varies between  0.02 to 0.04
##z₀ = αₛ*mean(Hm0)/2    # Charnock (1955)
@printf("z₀ estimate = %5.4f m\n",z₀)

U₁₀ = Uxx .* log(10/z₀) ./ log(xx/z₀)
U₁₀calc = Uz .* log(10/z₀) ./ log(1/z₀)
ma1 = movavg(Float64.(U₁₀calc), 1)

tm_tick = range(wind_sea_df.Date[1], wind_sea_df.Date[end], step=Hour(2))
ticks = Dates.format.(tm_tick,"HH:MM");

# Define x-limits
xlims = (wind_sea_df.Date[1],wind_sea_df.Date[end])

# Clip y-data based on x-limits
idx = (wind_data_df.Date .>= xlims[1]) .& (wind_data_df.Date .<= xlims[2])

p1 = Plots.plot(wind_data_df.Date[idx], U₁₀[idx] ./ 3.6, lc=:blue, lw=:3, alpha=0.75, label="U₁₀ observed\n", ylabel="Wind speed (m/s)")
p1 = Plots.plot!(wind_sea_df.Date, ma1.x, lc=:red, lw=:3, alpha=0.5, label="U₁₀ calc.")

##p1 = Plots.plot!(Plots.twinx(), wind_sea_df.Time_string, Hm0, lc=:green, label="", ylabel="Hm0 (m)")

plot_p1 = Plots.plot(p1, layout=(1,1), size=(1500,700), framestyle=:box, fg_legend=:transparent, bg_legend=:transparent, legendfontpointsize=:10,
    xticks=(tm_tick,ticks), xlims=(wind_sea_df.Date[1],wind_sea_df.Date[end]),
    leftmargin=15Plots.mm, rightmargin=15Plots.mm, gridlinewidth=0.5, gridstyle=:dot, gridalpha=1, 
    title=title, titlefontsize=:12)    

try
    # Output plot file name
    plt_file = ".\\Plots\\"*split(split(hxv_files[1],"\\")[end],"T")[1]*"_wind_comparison_plot.png" # <================== Change this as required!
    savefig(plot_p1, plt_file)
    println("\nPlot file saved as ",plt_file)
catch
    "Alert: Plot not saved!"
end

display(plot_p1)

In [ ]:
Uz = []

for i ∈ 1:nrow(wind_sea_df)
    
    frequency = wind_sea_df.Frequency[i] 
    spectra = wind_sea_df.Spectra[i]  

    uₛₜₐᵣ = mean(spectra .* frequency.^4) * (2π)^3 / (4*β*Ip*g)
    Uz_estimate = [uₛₜₐᵣ / κ * log(z / z₀) for z in 1:length(frequency)]
    
    push!(Uz, mean(Uz_estimate))

end
    
Plots.plot(merged_df.Date, merged_df.U₁₀ , lc=:blue, lw=:3, alpha=0.75, label="U₁₀ observed\n", ylabel="Wind speed (m/s)", size=(1500,700), xlims=(wind_sea_df.Date[1],wind_sea_df.Date[end]), ylims=(0,20))
Plots.plot!(wind_sea_df.Date, Uz, lc=:red, lw=:3, alpha=0.5, label="U₁₀ calc.")
Plots.plot!(wind_data_df.Date, wind_data_df.Wind_Speed ./ 3.6)


In [ ]:
wind_sea_df